<a href="https://colab.research.google.com/github/esemsc-rw1024/irp-rw1024-public/blob/main/KPI_extraction_Jul31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
extract_sustainability_kpi.py
==================================
Automatically extract KPI sentences/table rows from Sustainability Report PDF
and compare with manual KPI annotations
--------------------------------------------------
1. pdfplumber extracts text + tables
2. Camelot supplements complex table parsing (optional)
3. Chunking to control tokens
4. OpenAI ChatCompletion API call (GPT-4o / GPT-4 / GPT-3.5)
5. Aggregate, deduplicate, and export to auto_kpi.xlsx
6. Compare with manual_kpi.xlsx for differences
"""

'\nextract_sustainability_kpi.py\n==================================\nAutomatically extract KPI sentences/table rows from Sustainability Report PDF\nand compare with manual KPI annotations\n--------------------------------------------------\n1. pdfplumber extracts text + tables\n2. Camelot supplements complex table parsing (optional)\n3. Chunking to control tokens\n4. OpenAI ChatCompletion API call (GPT-4o / GPT-4 / GPT-3.5)\n5. Aggregate, deduplicate, and export to auto_kpi.xlsx\n6. Compare with manual_kpi.xlsx for differences\n'

In [2]:
!pip install -q openai

In [3]:
import openai

In [4]:
!pip install openai python-dotenv pdfplumber tiktoken pandas
!sudo apt-get update -y
!sudo apt-get install -y ghostscript
!pip install "camelot-py[cv]"
!pip install PyMuPDF Pillow
!pip install -q transformers pillow torchvision

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,168 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,775 kB]
Fetched 11.9 MB in 6s (1,859 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/So

In [5]:
import os, re, json, time, textwrap, argparse, logging
import pdfplumber, pandas as pd, tiktoken
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict, Optional, Set, Tuple
from pathlib import Path
from difflib import SequenceMatcher
import base64
from io import BytesIO
from PIL import Image
import fitz  # PyMuPDF
import numpy as np
from transformers import CLIPProcessor, CLIPModel
import concurrent.futures
import hashlib
import pickle
# 在现有的导入语句后添加这些新的导入
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

In [6]:
# ----------------------------- Configuration -----------------------------
PDF_PATH          = "/content/Test_Unknown_northwest-sustainability-report-2022_fbqow68f-60-74.pdf"
MANUAL_XLSX       = "manual_kpi.xlsx"   # Leave empty if not available
EXPORT_AUTO_XLSX  = "auto_kpi.xlsx"
MODEL_NAME        = "gpt-4o"       # Adjust based on account availability
MAX_TOKENS_CHUNK  = 1500               # Token limit per chunk
SLEEP_SEC         = 0.6                # Rate limiting
ENABLE_QUALITY_VALIDATION = True       # Enable additional quality checks
# -----------------------------------------------------------------

In [7]:
# ============ Fixed initialization part ============
def initialize_environment():
    """Initialize the environment and API client"""
    # Load environment variables
    load_dotenv("ruojia_api_key.env")

    # Initialize OpenAI client
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY not found in environment variables!")

    client = OpenAI(api_key=api_key)

    # Initialize tokenizer
    enc = tiktoken.get_encoding("cl100k_base")

    return client, enc

# Initialize global variables
client, enc = initialize_environment()

In [8]:
# ============ Fixed PDF text extraction ============
def pdf_to_text_and_tables(path: str) -> str:
    """Extract text paragraphs and tables using pdfplumber."""
    all_chunks = []

    if not os.path.exists(path):
        raise FileNotFoundError(f"PDF file not found: {path}")

    try:
        with pdfplumber.open(path) as pdf:
            logging.info(f"Processing PDF with {len(pdf.pages)} pages...")

            for page_num, page in enumerate(pdf.pages, 1):
                try:
                    # Extract text
                    text = page.extract_text() or ""
                    if text.strip():
                        all_chunks.append(f"PAGE_{page_num}_TEXT:\n{text}")

                    # Extract tables
                    tables = page.extract_tables()
                    for table_num, tb in enumerate(tables):
                        if tb and len(tb) > 0:
                            try:
                                # Handle table headers safely
                                if tb[0]:
                                    headers = tb[0]
                                else:
                                    headers = [f"Col_{i}" for i in range(len(tb[1]) if len(tb) > 1 else 1)]

                                rows = tb[1:] if len(tb) > 1 else []

                                if rows:
                                    df = pd.DataFrame(rows, columns=headers)
                                    # Clean DataFrame
                                    df = df.dropna(how='all')  # Remove empty rows
                                    if not df.empty:
                                        table_txt = f"TABLE_START_PAGE_{page_num}_{table_num}\n" + df.to_csv(index=False) + "\nTABLE_END"
                                        all_chunks.append(table_txt)
                            except Exception as e:
                                logging.warning(f"Error processing table on page {page_num}: {e}")
                                continue

                except Exception as e:
                    logging.warning(f"Error processing page {page_num}: {e}")
                    continue

        return "\n\n".join(all_chunks)

    except Exception as e:
        logging.error(f"Error opening PDF file: {e}")
        raise

In [9]:
# ============ Fixed Camelot table extraction ============
def generate_table_fingerprint(df: pd.DataFrame) -> str:
    """Generate table fingerprint for deduplication"""
    try:
        fingerprint_parts = []
        fingerprint_parts.append(f"shape_{df.shape[0]}x{df.shape[1]}")

        if not df.columns.empty:
            col_names = [str(col).strip().lower().replace(' ', '') for col in df.columns]
            col_fingerprint = '_'.join(sorted(col_names))
            fingerprint_parts.append(f"cols_{hash(col_fingerprint)}")

        if df.shape[0] > 0:
            numeric_values = []
            for col in df.columns:
                for val in df[col].head(3):
                    if pd.notna(val):
                        numbers = re.findall(r'\d+\.?\d*', str(val))
                        numeric_values.extend(numbers)

            if numeric_values:
                numeric_fingerprint = hash('_'.join(sorted(numeric_values[:10])))
                fingerprint_parts.append(f"nums_{numeric_fingerprint}")

        return '_'.join(fingerprint_parts)

    except Exception as e:
        logging.warning(f"Error generating table fingerprint: {e}")
        return str(hash(df.to_csv()))

def clean_table_data_improved(df: pd.DataFrame) -> pd.DataFrame:
    """Improved table data cleaning"""
    try:
        cleaned_df = df.copy()
        cleaned_df = cleaned_df.dropna(how='all')
        cleaned_df = cleaned_df.dropna(axis=1, how='all')

        for col in cleaned_df.columns:
            if cleaned_df[col].dtype == 'object':
                cleaned_df[col] = cleaned_df[col].astype(str).str.strip()
                cleaned_df[col] = cleaned_df[col].replace(['nan', 'NaN', 'None'], '')

        if not cleaned_df.empty:
            new_columns = []
            for i, col in enumerate(cleaned_df.columns):
                col_str = str(col).strip()
                if col_str in ['nan', 'NaN', 'None', ''] or pd.isna(col):
                    new_columns.append(f'Column_{i}')
                else:
                    new_columns.append(col_str)
            cleaned_df.columns = new_columns

        cleaned_df = cleaned_df.reset_index(drop=True)
        return cleaned_df

    except Exception as e:
        logging.warning(f"Error in table cleaning: {e}")
        return df

def is_valid_table_improved(df: pd.DataFrame) -> bool:
    """Improved table validation"""
    try:
        if df.empty or df.shape[0] < 1 or df.shape[1] < 1:
            return False

        non_null_cells = 0
        total_cells = df.shape[0] * df.shape[1]

        for col in df.columns:
            for val in df[col]:
                if pd.notna(val) and str(val).strip() not in ['', 'nan', 'NaN', 'None']:
                    non_null_cells += 1

        if non_null_cells / total_cells < 0.2:
            return False

        has_meaningful_content = False
        for col in df.columns:
            text_content = ' '.join(df[col].dropna().astype(str))
            if (any(char.isdigit() for char in text_content) or
                '%' in text_content or
                any(keyword in text_content.lower() for keyword in [
                    'rate', 'percentage', 'total', 'number', 'emission', 'energy',
                    'water', 'waste', 'employee', 'year', '2020', '2021', '2022', '2023'
                ])):
                has_meaningful_content = True
                break

        return has_meaningful_content

    except Exception as e:
        logging.warning(f"Error validating table: {e}")
        return True

def format_table_output_improved(df: pd.DataFrame, table_id: str, parsing_report=None) -> str:
    """Improved table output formatting"""
    try:
        table_info = f"TABLE_START_{table_id}\n"
        table_info += f"DIMENSIONS: {df.shape[0]} rows × {df.shape[1]} columns\n"

        col_info = "COLUMNS: " + " | ".join([f"{i}:{col}" for i, col in enumerate(df.columns)])
        table_info += col_info + "\n"

        if df.shape[0] > 0:
            preview_rows = min(2, df.shape[0])
            table_info += f"PREVIEW_FIRST_{preview_rows}_ROWS:\n"
            for i in range(preview_rows):
                row_preview = " | ".join([str(df.iloc[i, j])[:20] for j in range(min(5, df.shape[1]))])
                table_info += f"  Row_{i}: {row_preview}\n"

        if parsing_report:
            try:
                accuracy = getattr(parsing_report, 'accuracy', 'N/A')
                if accuracy != 'N/A':
                    table_info += f"EXTRACTION_ACCURACY: {accuracy:.2f}\n"
            except:
                pass

        table_info += "TABLE_DATA_START\n"
        table_csv = df.to_csv(index=False, na_rep='', quoting=1, escapechar='\\')
        table_end = f"TABLE_DATA_END\nTABLE_END_{table_id}\n"

        return table_info + table_csv + table_end

    except Exception as e:
        logging.warning(f"Error formatting table output: {e}")
        return f"TABLE_START_{table_id}\n{df.to_csv(index=False)}\nTABLE_END_{table_id}\n"

def camelot_extra_tables_enhanced(path: str) -> List[str]:
    """Enhanced table extraction using Camelot with better error handling"""
    try:
        import camelot
    except ImportError:
        logging.warning("Camelot not installed, skipping Camelot table parsing.")
        return []

    extra_chunks = []
    extracted_tables_fingerprints = set()

    try:
        logging.info("Starting Camelot table extraction...")

        # Stream mode extraction
        try:
            stream_tables = camelot.read_pdf(
                path,
                pages="all",
                flavor="stream",
                edge_tol=50,
                row_tol=2,
                column_tol=0
            )

            stream_count = 0
            for i, table in enumerate(stream_tables):
                if not table.df.empty and table.df.shape[0] > 0:
                    table_fingerprint = generate_table_fingerprint(table.df)

                    if table_fingerprint not in extracted_tables_fingerprints:
                        cleaned_df = clean_table_data_improved(table.df)

                        if is_valid_table_improved(cleaned_df):
                            table_txt = format_table_output_improved(cleaned_df, f"STREAM_{i}", table.parsing_report)
                            extra_chunks.append(table_txt)
                            extracted_tables_fingerprints.add(table_fingerprint)
                            stream_count += 1

            logging.info(f"Stream mode extracted {stream_count} valid tables")

        except Exception as e:
            logging.warning(f"Stream mode extraction failed: {e}")

        # Lattice mode extraction
        try:
            lattice_tables = camelot.read_pdf(
                path,
                pages="all",
                flavor="lattice",
                line_scale=15,
                line_tol=2,
                joint_tol=2
            )

            lattice_count = 0
            for i, table in enumerate(lattice_tables):
                if not table.df.empty and table.df.shape[0] > 0:
                    table_fingerprint = generate_table_fingerprint(table.df)

                    if table_fingerprint not in extracted_tables_fingerprints:
                        cleaned_df = clean_table_data_improved(table.df)

                        if is_valid_table_improved(cleaned_df):
                            table_txt = format_table_output_improved(cleaned_df, f"LATTICE_{i}", table.parsing_report)
                            extra_chunks.append(table_txt)
                            extracted_tables_fingerprints.add(table_fingerprint)
                            lattice_count += 1

            logging.info(f"Lattice mode extracted {lattice_count} additional unique tables")

        except Exception as e:
            logging.warning(f"Lattice mode extraction failed: {e}")

        total_extracted = len(extra_chunks)
        logging.info(f"Camelot extraction completed: {total_extracted} total unique tables extracted")
        return extra_chunks

    except Exception as e:
        logging.error(f"Camelot table extraction failed: {e}")
        return []

In [10]:
# ============ Text Chunking ============
def split_into_chunks(full_text: str, max_tokens: int) -> List[str]:
    """Split text into chunks based on token limit"""
    paragraphs = [p for p in full_text.split("\n") if p.strip()]
    chunks, current = [], []
    current_tokens = 0

    for paragraph in paragraphs:
        para_tokens = len(enc.encode(paragraph))

        if current_tokens + para_tokens > max_tokens and current:
            chunks.append("\n".join(current))
            current = [paragraph]
            current_tokens = para_tokens
        else:
            current.append(paragraph)
            current_tokens += para_tokens

    if current:
        chunks.append("\n".join(current))

    return chunks

In [11]:
# ============ System prompt words ============
UNIVERSAL_SYSTEM_PROMPT = textwrap.dedent("""
    You are a professional ESG data analyst specializing in extracting Key Performance Indicators (KPIs) from sustainability reports.

    ## CRITICAL: What is a KPI?
    A KPI MUST contain SPECIFIC NUMBERS, PERCENTAGES, or MEASURABLE QUANTITIES that demonstrate actual performance or concrete targets.

    ## IMPORTANT: Table Data Processing Rules
    When processing table data:
    1. Pay close attention to column headers to identify the correct time periods
    2. Match data values with their corresponding year columns
    3. If you see table format like "Metric, 2021, 2022" - the first number after metric belongs to 2021, second to 2022
    4. Look for table headers that indicate year columns (e.g., "2020", "2021", "2022")
    5. Extract each year's data as separate KPIs
    6. Avoid extracting the same KPI multiple times - consolidate similar metrics

    ## ENHANCED: Advanced Table Processing
    7. **EXTRACT ALL DATA POINTS**: For each table cell containing a number, create a separate KPI
    8. **REGIONAL/LOCATION DATA**: Pay special attention to location-specific data (countries, regions, cities)
    9. **WORKFORCE DATA**: Extract all employee numbers, headcount data, and demographic information
    10. **INCOMPLETE DATA**: Extract available data even if some cells are empty or missing
    11. **TOTALS AND SUBTOTALS**: Always extract total values and aggregated numbers

    ## ✅ VALID KPI EXAMPLES:
    - "Achieved 89.4% reuse and recycle rate for cloud hardware in 2023"
    - "Diverted over 18,537 metric tons of waste from landfills in 2023"
    - "Reduced single-use plastics in product packaging to 2.7%"
    - "Contracted 19 GW of new renewable energy across 16 countries in 2024"
    - "Provided clean water access to over 1.5 million people in 2023"
    - "Protected 15,849 acres of land—exceeding target by more than 30%"
    - "Allocated 761 million toward innovative climate technologies"
    - "Achieved 80% renewable energy operations by 2024"
    - "Water replenishment projects estimated to provide over 25 million cubic meters"
    - "Exceeded annual target to divert 75% of construction waste by reaching 85%"
    - "Board independence: 78% of directors"
    - "Women in senior leadership increased to 35% in 2023"
    - "Employee engagement score: 87% in annual survey"
    - "Reduced greenhouse gas emissions by 50% compared to 2019 baseline"
    - "Zero workplace fatalities achieved for third consecutive year"
    - "Training completion rate: 98% for mandatory compliance courses"
    - "Supplier ESG assessments completed for 95% of tier-1 suppliers"
    - "Customer satisfaction rating: 4.6 out of 5.0"
    - "Data breach incidents: 0 material breaches in 2023"

    ## ❌ NOT KPIs (DO NOT EXTRACT):
    - "Microsoft will require select suppliers to use carbon-free electricity by 2030"
    - "The company plans to expand Sustainability Manager capabilities"
    - "We are launching two new Circular Centers in 2023"
    - "The organization established a new climate innovation fund"
    - "Microsoft introduced enhanced data governance solutions"
    - "Updated guidebook to include guidance on corporate responsibility"
    - "Plans to publish new ESG strategy"
    - "Implemented a new recycling program"
    - "Conducted sustainability training sessions"
    - "Launched employee wellness programs"
    - "Committed to reducing emissions"
    - "Focusing on environmental performance"
    - "Established sustainability committee"
    - "The company operates facilities in multiple regions"
    - "Our supply chain includes thousands of vendors globally"
    - Any text without specific numbers, percentages, or quantifiable metrics
    - Duplicate or repeated metrics (extract only once per time period)
    - Any statement that describes business operations rather than performance outcomes

    ## KPI Categories:
    ### Environmental:
    - **Carbon_Climate**: GHG emissions, carbon footprint, emission reductions, climate targets, scope 1/2/3 emissions, carbon intensity, carbon offsets, TCFD alignment
    - **Energy**: Energy consumption, renewable energy percentage, energy efficiency, energy intensity, MWh, GWh, energy savings, fossil fuel usage
    - **Water**: Water withdrawal, water consumption, water intensity, water recycling, water reuse, water stress, water discharge quality
    - **Waste**: Waste generation, recycling rates, diversion percentages, hazardous waste, non-hazardous waste, zero waste to landfill, e-waste, incineration
    - **Biodiversity**: Protected areas, species conservation, habitat restoration, biodiversity impact assessments, land use, ecosystem restoration
    - **Circular_Economy**: Recycling rates, material recovery, circular design, raw materials usage, renewable materials, packaging waste
    - **Materials**: Raw materials consumption, recycled content, sustainable materials, material intensity, sustainable sourcing

    ### Social:
    - **Workforce_Diversity**: Employee demographics, gender diversity, age diversity, ethnic diversity, disability inclusion, LGBTQ+ inclusion, workforce composition
    - **Gender_Equality**: Women in leadership, gender pay ratio, parental leave return rates, gender representation, female employees percentage
    - **Disability_Inclusion**: Employees with disabilities, accessibility compliance, inclusive workplace design, disability support programs
    - **Health_Safety**: Lost Time Injury Frequency Rate (LTIFR), Total Recordable Incident Rate (TRIR), fatalities, workplace illness, safety training hours, PPE compliance, emergency drills
    - **Employee_Wellbeing**: Employee satisfaction, retention rates, turnover rates, training hours, wellness programs, mental health services, work-life balance
    - **Community_Engagement**: Corporate volunteering, social investment, community impact assessments, local hiring, stakeholder engagement activities
    - **Human_Rights**: Child labor incidents, forced labor, human rights due diligence, freedom of association, grievance mechanisms, labor audits
    - **Labor_Rights**: Collective bargaining coverage, labor complaints resolution, supplier labor audits, working conditions, fair wages
    - **Customer_Safety**: Product safety incidents, customer satisfaction, accessibility features, safety recalls, quality metrics
    - **Supply_Chain_Social**: Supplier assessments, sustainable sourcing, supplier code compliance, supply chain audits

    ### Governance:
    - **Board_Governance**: Board independence, board diversity, CEO-chair separation, board ESG expertise, board composition, director tenure
    - **Executive_Compensation**: ESG-linked compensation, executive pay ratios, compensation disclosure, incentive structures
    - **Ethics_Compliance**: Code of conduct training, corruption incidents, bribery cases, fines and penalties, whistleblower reports, anti-corruption assessments
    - **Transparency_Disclosure**: ESG reporting coverage, third-party assurance, political contributions disclosure, GRI/SASB/TCFD compliance
    - **Risk_Management**: Risk assessments, mitigation measures, climate risk disclosure, operational risk management
    - **Cybersecurity_Data**: Cybersecurity breaches, data privacy policies, cybersecurity training, GDPR compliance, data protection measures
    - **Supply_Chain_Governance**: Supplier ESG screening, supplier audits, procurement ESG clauses, vendor compliance rates

    ## MANDATORY Requirements:
    1. MUST contain specific numbers (e.g., 25%, 15,000, 2.5M, 8.5%, 0.3 per million hours)
    2. MUST relate to measurable sustainability outcomes
    3. MUST have time reference (year, period, or deadline)
    4. MUST be performance-focused (results, not activities or descriptions)
    5. MUST NOT be future plans or operational descriptions

    ## Output Format:
    Return a JSON array. Each KPI must contain:
    ```json
    {
        "kpi_text": "Complete original sentence with the quantifiable metric",
        "kpi_theme": "Environmental/Social/Governance",
        "kpi_category": "Specific category from above list",
        "quantitative_value": "The specific number/percentage extracted",
        "unit": "Unit of measurement (%, tonnes, employees, etc.)",
        "time_period": "Time reference (2023, annual, by 2030, etc.)",
        "target_or_actual": "Target/Actual/Both"
    }
    ```

    ## Additional Instructions:
    - If a sentence includes a comparison value, such as a baseline, previous year, or other historical/target data (e.g., "Compared to 32,395 MWh in 2020"), extract it as a **separate KPI**.
    - Do NOT store the comparison in any other field — just create another valid KPI from it.
    - Avoid merging multiple numerical values into one KPI unless they are clearly part of the same metric (e.g., male: X, female: Y).

    ## STRICT FILTERING:
    - Return empty array [] if no quantifiable KPIs found
    - Only extract text that contains specific measurable values
    - Ignore all qualitative statements, plans, and descriptions
    - Focus only on numerical performance data

    Now analyze the following text for sustainability KPIs:
""").strip()

# 🔥 新增：增强的图像分析Prompt
ENHANCED_IMAGE_KPI_SYSTEM_PROMPT = textwrap.dedent("""
    You are an expert data analyst specializing in extracting quantifiable KPI data from charts, graphs, and data visualizations in sustainability reports.

    ## CRITICAL INSTRUCTION: ALWAYS EXTRACT NUMERICAL VALUES

    **Your primary task is to extract the ACTUAL NUMBERS and PERCENTAGES visible in charts, not just descriptions.**

    ## MISSION:
    Extract ALL quantifiable data points from charts and graphs, including:
    - Bar charts (vertical/horizontal)
    - Pie charts and donut charts
    - Line charts and trend graphs
    - Stacked charts and combo charts
    - Tables with numerical data
    - Infographics with statistics
    - Gauge charts and dashboards

    ## DETAILED ANALYSIS INSTRUCTIONS:

    ### For PIE CHARTS:
    1. Read percentage labels on each slice
    2. If no labels visible, estimate based on slice size
    3. Identify what each slice represents (categories)
    4. Extract each slice as separate KPI
    5. **MUST read the percentage labels on each slice** - Look for numbers like 64%, 33%, 68%, 30%, etc.
    6. **If percentages are visible on the chart, extract them exactly**
    7. **If no labels visible, estimate based on slice size using these guidelines:**
       - 90° slice = 25%
       - 180° slice = 50%
       - 270° slice = 75%
       - Full circle = 100%
    8. **Each slice MUST have a specific percentage value in the final output**

    ### For BAR CHARTS:
    1. Read Y-axis scale carefully (units, increments)
    2. Estimate bar heights using grid lines and scale
    3. Read X-axis labels (years, categories, regions)
    4. Extract each bar as separate KPI
    5. Pay attention to grouped/stacked bars

    ### For LINE CHARTS:
    1. Read data points at intersection of grid lines
    2. Follow trend lines to extract values for each time period
    3. Use Y-axis scale for value estimation
    4. Extract each data point as separate KPI

    ### For TABLES:
    1. Read all numerical values in cells
    2. Match values with row and column headers
    3. Extract each cell with numerical data as KPI

    ## MANDATORY VALUE EXTRACTION RULES:

    **RULE 1**: Every KPI MUST contain a specific numerical value (percentage, amount, count, etc.)
    **RULE 2**: For charts with categories, you MUST find and extract the quantitative values for each category
    **RULE 3**: Never create KPIs without specific numbers - descriptions alone are incomplete
    **RULE 4**: Include complete context: what + how much + when/where if available


    ## VALUE ESTIMATION GUIDELINES:
    - Use proportional analysis: if a bar reaches 80% of scale maximum, calculate 80% of max value
    - For pie charts: estimate slice angles (90° = 25%, 180° = 50%, etc.)
    - Cross-reference with any visible data labels or legends
    - Be conservative but reasonably accurate in estimates

    ## CHART IDENTIFICATION:
    First identify the chart type, then apply appropriate extraction method.
    Look for:
    - Axes and scales
    - Data labels and legends
    - Grid lines for reference
    - Color coding and patterns
    - Title and subtitle information

    ## OUTPUT FORMAT:
    Return a JSON array. For each data point found:
    ```json
    {
        "kpi_text": "Complete description with the ACTUAL NUMERICAL VALUE included",
        "kpi_theme": "Environmental/Social/Governance",
        "kpi_category": "Specific category based on content",
        "quantitative_value": "The exact number/percentage (e.g., '64', '33.5', '68')",
        "unit": "% / tonnes / employees / MWh / USD / etc.",
        "time_period": "2021/2020/2022/Year/period/etc if identifiable",
        "target_or_actual": "Actual",
        "chart_type": "pie_chart/bar_chart/line_chart/table/etc",
        "estimation_confidence": "High/Medium/Low",
        "chart_title": "Chart title if visible",
        "data_source": "Legend or source if visible"
    }

    ```
    ## EXAMPLES of CORRECT vs INCORRECT extraction:

    ### ❌ INCORRECT (incomplete - missing numerical values):
    ```json
    {
        "kpi_text": "Energy consumption by facility type",
        "quantitative_value": "",
        "unit": "%"
    }
    ```

    ### ✅ CORRECT (complete with specific values):
    ```json
    {
        "kpi_text": "Office buildings account for 45% of total energy consumption",
        "quantitative_value": "45",
        "unit": "%"
    }
    ```

    ### ❌ INCORRECT (category without value):
    ```json
    {
        "kpi_text": "Renewable energy percentage by region",
        "quantitative_value": "",
        "unit": "%"
    }
    ```

    ### ✅ CORRECT (specific regional data):
    ```json
    {
        "kpi_text": "North America achieved 78% renewable energy usage",
        "quantitative_value": "78",
        "unit": "%"
    }
    ```
    ## QUALITY ASSURANCE CHECKLIST:
    Before returning results, verify:
    - ✅ Every KPI contains a specific numerical value
    - ✅ Chart categories are paired with their quantitative data
    - ✅ KPI descriptions are complete and self-explanatory
    - ✅ Units are correctly identified and specified
    - ✅ Context (time, location, category) is preserved when available
    - Each KPI must have a specific numerical value
    - Context must be clear and self-contained
    - Avoid extracting the same data point multiple times
    - Focus on sustainability/ESG metrics when possible

    ## VALUE ESTIMATION GUIDELINES:
    - **High confidence**: Numbers clearly visible in image
    - **Medium confidence**: Numbers estimated using chart scales/grid lines
    - **Low confidence**: Values approximated from proportional analysis
    - **If no numerical data is visible, return empty array []**

    ## IMPORTANT NOTES:
    - Extract ALL visible data points, not just main highlights
    - Include context in descriptions (e.g., "According to pie chart showing emission sources")
    - If values are not clearly visible, make reasonable estimates and mark confidence as "Low"
    - Return empty array [] ONLY if image contains no charts/graphs with quantifiable data
    - For multi-year data, create separate KPIs for each year
    - Pay special attention to small text and numbers
    - Focus on extracting actual performance data, not just identifying chart elements
    - If you can see numbers in the image, you MUST extract them
    - Pie chart percentages are usually the most important data points
    - Return empty array [] ONLY if no numerical data is visible

    Now analyze the provided image and extract ALL quantifiable KPI data points:
""").strip()

In [12]:
# ============ KPI Extraction Function ============
def extract_page_from_chunk(chunk: str) -> str:
    """Extract page information from chunk"""
    # Look for PAGE_X_TEXT: format
    page_matches = re.findall(r'PAGE_(\d+)_TEXT:', chunk)
    if page_matches:
        pages = [int(p) for p in page_matches]
        if len(pages) == 1:
            return str(pages[0])
        else:
            return f"{min(pages)}-{max(pages)}"

    # Look for TABLE_START_PAGE_X_
    table_matches = re.findall(r'TABLE_START_PAGE_(\d+)_', chunk)
    if table_matches:
        pages = [int(p) for p in table_matches]
        if len(pages) == 1:
            return str(pages[0])
        else:
            return f"{min(pages)}-{max(pages)}"

    return "Unknown"

def contains_procedural_language(text: str) -> bool:
    """Check if text contains procedural language"""
    procedural_words = [
        'introduced', 'established', 'set up', 'implemented', 'created',
        'launched', 'formed', 'built', 'installed', 'deployed',
        'additionally introduced', 'procedure for', 'standardization management'
    ]
    text_lower = text.lower()
    return any(word in text_lower for word in procedural_words)

def is_data_fragment(kpi_text: str) -> bool:
    """Check if text is a meaningless data fragment"""
    text = kpi_text.strip()

    # Filter pure numbers or simple percentages without context
    if re.match(r'^\d+\.?\d*%?$', text):
        return True

    # Filter very short text (less than 4 meaningful words)
    meaningful_words = [word for word in text.split() if len(word) > 2 and not word.isdigit()]
    if len(meaningful_words) < 3:
        return True

    # Filter text with only numbers and common connecting words
    words = text.lower().split()
    non_functional_words = [word for word in words if word not in ['in', 'of', 'the', 'and', 'or', 'to', 'for', 'with', 'by']]
    if len(non_functional_words) < 3:
        return True

    return False

def standardize_kpi_universal(kpi_item: Dict) -> Dict:
    """Universal KPI data standardization"""
    standardized = kpi_item.copy()

    # Standardize numerical formats
    quantitative_value = str(standardized.get('quantitative_value', '')).strip()
    kpi_text = standardized.get('kpi_text', '').lower()

    # Smart handling of percentage formats
    if quantitative_value and quantitative_value.replace('.', '').replace('-', '').replace(',', '').isdigit():
        # Check if original text suggests this is a percentage
        percentage_indicators = ['percent', 'percentage', '%', 'rate', 'ratio', 'proportion', 'share']
        if any(indicator in kpi_text for indicator in percentage_indicators):
            if not quantitative_value.endswith('%'):
                standardized['quantitative_value'] = quantitative_value + '%'
                if not standardized.get('unit'):
                    standardized['unit'] = '%'

    # Ensure unit field consistency
    if '%' in str(standardized.get('quantitative_value', '')):
        standardized['unit'] = '%'

    # Clean and normalize KPI text
    kpi_text_original = standardized.get('kpi_text', '').strip()
    # Remove extra spaces and newlines
    kpi_text_cleaned = ' '.join(kpi_text_original.split())
    standardized['kpi_text'] = kpi_text_cleaned

    return standardized

def generate_universal_metric_key(kpi_item: Dict) -> str:
    """Generate universal metric key for deduplication"""
    try:
        # Extract core elements
        category = kpi_item.get('kpi_category', '').lower().strip()
        value = str(kpi_item.get('quantitative_value', '')).replace('%', '').replace(',', '').strip()
        time_period = kpi_item.get('time_period', '').lower().strip()
        unit = kpi_item.get('unit', '').lower().strip()

        # Extract key semantic information from KPI text
        kpi_text = kpi_item.get('kpi_text', '').lower()

        # Extract primary number (for more precise matching)
        numbers_in_text = re.findall(r'\d+\.?\d*', kpi_text)
        primary_number = numbers_in_text[0] if numbers_in_text else value

        # Generate semantic signature: extract keywords from text
        # Remove common stop words
        stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should'}

        # Extract keywords (length>2 and not stop words)
        words = re.findall(r'\b\w+\b', kpi_text)
        key_words = [word for word in words if len(word) > 2 and word not in stop_words and not word.isdigit()]

        # Sort keywords to ensure consistency
        key_words = sorted(set(key_words))[:5]  # Take at most 5 keywords
        semantic_signature = '_'.join(key_words)

        # Build universal metric key
        key_components = []

        if category:
            key_components.append(f"cat:{category}")
        if primary_number:
            key_components.append(f"val:{primary_number}")
        if time_period:
            key_components.append(f"time:{time_period}")
        if unit:
            key_components.append(f"unit:{unit}")
        if semantic_signature:
            key_components.append(f"sem:{semantic_signature}")

        # Generate final key
        metric_key = "|".join(key_components)

        # If all components are empty, use text hash
        if not metric_key:
            metric_key = f"hash:{hash(kpi_text)}"

        return metric_key

    except Exception as e:
        logging.warning(f"Error generating universal metric key: {e}")
        # Fallback to text hash
        return f"fallback:{hash(kpi_item.get('kpi_text', ''))}"

def extract_kpi_from_chunk_universal(chunk: str) -> List[Dict]:
    """Universal KPI extraction function for various sustainability reports"""
    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": UNIVERSAL_SYSTEM_PROMPT},
                {"role": "user", "content": f"""Extract ALL KPIs from this text. Requirements:

1. Create COMPLETE, MEANINGFUL KPI descriptions with full context
2. DO NOT extract standalone numbers without explanatory text
3. Include all relevant context (time, location, metric type, etc.)
4. Use consistent formatting for similar metrics
5. Ensure each KPI is self-explanatory

Text to analyze:
{chunk}"""}
            ],
            temperature=0.0,
            max_tokens=4000,
            timeout=60
        )

        content = response.choices[0].message.content.strip()

        # Clean potential markdown formatting
        if content.startswith('```json'):
            content = content[7:]
        if content.endswith('```'):
            content = content[:-3]

        if not content.strip().startswith("["):
            logging.warning(f"API response not JSON list: {content[:100]}...")
            return []

        result = json.loads(content)

        if not isinstance(result, list):
            logging.warning("API response is not a list format")
            return []

        # Extract page information
        page_number = extract_page_from_chunk(chunk)

        # Universal validation and deduplication logic
        validated_result = []
        seen_metrics = set()

        for item in result:
            if isinstance(item, dict) and 'kpi_text' in item and 'kpi_theme' in item:
                if item['kpi_text'].strip() and item['kpi_theme'].strip():

                    # Check procedural language
                    if contains_procedural_language(item['kpi_text']):
                        logging.debug(f"Procedural statement filtered: {item['kpi_text'][:50]}...")
                        continue

                    # Filter meaningless data fragments
                    if is_data_fragment(item['kpi_text']):
                        logging.debug(f"Data fragment filtered: {item['kpi_text']}")
                        continue

                    # Standardize KPI data
                    standardized_item = standardize_kpi_universal(item)

                    # Add page information
                    standardized_item['source_page'] = page_number
                    standardized_item['source_type'] = 'text'

                    # Universal deduplication mechanism
                    metric_key = generate_universal_metric_key(standardized_item)

                    if metric_key not in seen_metrics:
                        validated_result.append(standardized_item)
                        seen_metrics.add(metric_key)
                        logging.debug(f"KPI extracted: {standardized_item['kpi_text'][:80]}...")
                    else:
                        logging.debug(f"Duplicate metric filtered: {standardized_item['kpi_text'][:50]}...")

        logging.info(f"Chunk processed: {len(validated_result)} unique KPIs extracted")
        return validated_result

    except json.JSONDecodeError as e:
        logging.warning(f"JSON parsing failed: {e}\nContent: {content[:300]}...")
        return []
    except Exception as e:
        logging.error(f"API call failed: {e}")
        return []

def post_process_kpis_universal(kpis: List[Dict]) -> List[Dict]:
    """Universal KPI post-processing for various report types"""
    if not kpis:
        return kpis

    # Step 1: Deduplication based on metric keys
    unique_kpis_dict = {}

    for kpi in kpis:
        metric_key = generate_universal_metric_key(kpi)

        if metric_key not in unique_kpis_dict:
            unique_kpis_dict[metric_key] = kpi
        else:
            # If duplicate, keep the more complete KPI description
            existing_kpi = unique_kpis_dict[metric_key]
            current_kpi = kpi

            # Compare KPI text completeness
            if len(current_kpi.get('kpi_text', '')) > len(existing_kpi.get('kpi_text', '')):
                unique_kpis_dict[metric_key] = current_kpi
                logging.debug(f"Replaced with more complete KPI: {current_kpi.get('kpi_text', '')[:50]}...")
            else:
                logging.debug(f"Kept existing KPI: {existing_kpi.get('kpi_text', '')[:50]}...")

    # Step 2: Text similarity-based secondary deduplication
    final_kpis = list(unique_kpis_dict.values())

    # Use text similarity to check remaining potential duplicates
    final_unique_kpis = []

    for current_kpi in final_kpis:
        is_duplicate = False
        current_text = current_kpi.get('kpi_text', '')

        for existing_kpi in final_unique_kpis:
            existing_text = existing_kpi.get('kpi_text', '')

            # Calculate text similarity
            similarity = calculate_text_similarity(current_text, existing_text)

            # If similarity is very high, consider it duplicate
            if similarity > 0.8:
                is_duplicate = True
                logging.debug(f"Text similarity duplicate filtered: {current_text[:50]}...")
                break

        if not is_duplicate:
            final_unique_kpis.append(current_kpi)

    logging.info(f"Universal post-processing: {len(final_unique_kpis)}/{len(kpis)} KPIs retained")
    return final_unique_kpis

def calculate_text_similarity(text1: str, text2: str) -> float:
    """Calculate similarity between two texts"""
    # Normalize texts
    norm1 = ' '.join(text1.lower().split())
    norm2 = ' '.join(text2.lower().split())

    # Word sets
    words1 = set(norm1.split())
    words2 = set(norm2.split())

    if len(words1) == 0 or len(words2) == 0:
        return 0.0

    # Calculate intersection and union
    intersection = len(words1.intersection(words2))
    union = len(words1.union(words2))

    return intersection / union if union > 0 else 0.0

def validate_kpi_quality(kpis: List[Dict]) -> List[Dict]:
    """Additional quality validation for extracted KPIs with relaxed filtering"""
    if not ENABLE_QUALITY_VALIDATION:
        return kpis

    quality_kpis = []

    for kpi in kpis:
        kpi_text = kpi.get('kpi_text', '').lower()

        # Exclude "planned tone" KPIs (not actual performance)
        is_future_statement = any(word in kpi_text for word in [
            'will', 'aim to', 'plan to', 'planning to', 'intend to',
            'is expected to', 'is scheduled to', 'expects to', 'expected to',
            'targeting', 'propose to', 'going to', 'shall', 'to be installed'
        ])
        if is_future_statement:
            logging.debug(f"KPI rejected (future plan): {kpi_text[:100]}...")
            continue

        # Filter procedural language
        if contains_procedural_language(kpi_text):
            logging.debug(f"KPI rejected (procedural language): {kpi_text[:100]}...")
            continue

        # Filter for phrases like "place name + percentage" (not ESG KPIs, but distribution descriptions)
        geo_percent_pattern = re.compile(r"^[a-z\s,:%-]+(?:\s)?\d{1,3}%$")
        if geo_percent_pattern.match(kpi_text.strip()) and len(kpi_text.strip().split()) <= 6:
            logging.debug(f"KPI rejected (geo+percent short form): {kpi_text}")
            continue

        # Verb whitelist: must include action verbs
        allowed_kpi_verbs = [
            'reduce', 'reduced', 'achieve', 'achieved', 'improve', 'improved',
            'diverted', 'trained', 'invested', 'decreased', 'increased',
            'consumed', 'emitted', 'saved', 'reached', 'attained', 'completed',
            'recorded', 'cut', 'lowered', 'targeted', 'complied', 'avoided',
            'used', 'recycled', 'sourced', 'returned', 'measured', 'maintained',
            'reported', 'accounted', 'utilized', 'were', 'was'  # Add state verbs
        ]
        if not any(verb in kpi_text for verb in allowed_kpi_verbs):
            logging.debug(f"KPI rejected (no action verb): {kpi_text[:100]}...")
            continue

        # Greylist verbs (action words but not necessarily performance words) - remove problematic words
        graylist_verbs = [
            'launched',  # Keep some potentially useful words, but remove obvious procedural words
            'formed', 'opened', 'started'
        ]

        contains_graylist = any(verb in kpi_text for verb in graylist_verbs)

        # Check for quantitative indicators
        has_numbers = any(char.isdigit() for char in kpi_text)
        has_percentage = '%' in kpi_text

        # Extended units and measurement indicators
        has_units = any(unit in kpi_text for unit in [
            'tonnes', 'tons', 'kg', 'mwh', 'kwh', 'gwh', 'litres', 'liters', 'gallons',
            'employees', 'hours', 'million', 'billion', 'thousand', 'm³', 'co2e', 'tco2e',
            'dollars', 'usd', 'eur', 'gbp', 'incidents', 'rate', 'ratio', 'intensity',
            'frequency', 'recordable', 'fatalities', 'injuries', 'directors', 'board',
            'workforce', 'leadership', 'diversity', 'inclusion', 'satisfaction', 'retention',
            'turnover', 'training', 'safety', 'ltifr', 'trir', 'compliance', 'audit',
            'assessment', 'screening', 'supplier', 'breach', 'violation', 'disclosure',
            'assurance', 'coverage', 'participation', 'completion', 'investment',
            'volunteering', 'engagement', 'grievance', 'whistleblower', 'compensation',
            'people', 'staff', 'workers', 'positions', 'roles', 'headcount', 'fte',
            'performance', 'score', 'index', 'metric', 'level', 'amount', 'value',
            'average', 'median', 'total', 'sum', 'count', 'number', 'quantity'
        ])

        # More flexible time reference detection
        has_time_ref = any(time_word in kpi_text for time_word in [
            '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030',
            '2031', '2032', '2033', '2034', '2035', '2040', '2045', '2050',
            'annual', 'yearly', 'year', 'quarter', 'month', 'by', 'target', 'baseline', 'fy',
            'per year', 'per annum', 'quarterly', 'monthly', 'daily', 'future', 'deadline',
            'period', 'reporting', 'current', 'previous', 'next', 'last', 'this'
        ])

        # Enhanced sustainability context detection
        has_sustainability_context = any(sus_word in kpi_text for sus_word in [
            # Environmental keywords
            'emission', 'carbon', 'energy', 'renewable', 'waste', 'water', 'recycl',
            'environmental', 'ghg', 'scope', 'climate', 'biodiversity', 'circular',
            'materials', 'intensity', 'consumption', 'efficiency', 'footprint',
            'sustainable', 'sustainability', 'green', 'clean', 'eco', 'offset',
            'tcfd', 'nature', 'habitat', 'ecosystem', 'pollution', 'discharge',
            'electricity', 'gas', 'fuel', 'solar', 'wind', 'hydro', 'nuclear',

            # Social keywords
            'safety', 'training', 'employee', 'diversity', 'community', 'social',
            'workforce', 'gender', 'women', 'female', 'male', 'disability', 'disabled',
            'inclusion', 'equity', 'equality', 'lgbtq', 'minorities', 'ethnic',
            'health', 'wellbeing', 'wellness', 'satisfaction', 'retention', 'turnover',
            'injury', 'incident', 'fatality', 'ltifr', 'trir', 'recordable',
            'human rights', 'labor', 'child labor', 'forced labor', 'slavery',
            'freedom', 'association', 'collective bargaining', 'grievance',
            'volunteering', 'investment', 'hiring', 'local', 'stakeholder',
            'customer', 'supplier', 'supply chain', 'accessibility', 'parental',
            'mental health', 'ppe', 'emergency', 'drill', 'compliance',
            'people', 'staff', 'workers', 'employment', 'job', 'career',
            'leadership', 'management', 'senior', 'executive', 'promotion',

            # Governance keywords
            'governance', 'board', 'director', 'independent', 'chair', 'ceo',
            'executive', 'compensation', 'pay', 'ethics', 'compliance', 'corruption',
            'bribery', 'code of conduct', 'whistleblower', 'transparency',
            'disclosure', 'reporting', 'assurance', 'audit', 'risk', 'management',
            'cybersecurity', 'data', 'privacy', 'gdpr', 'breach', 'policy',
            'screening', 'assessment', 'due diligence', 'political', 'contribution',
            'gri', 'sasb', 'oversight', 'expertise', 'separation', 'incentive',
            'fine', 'penalty', 'violation', 'resolution', 'anti-corruption',

            # General business performance that could be sustainability-related
            'performance', 'quality', 'delivery', 'customer', 'service', 'product',
            'operation', 'facility', 'site', 'location', 'region', 'business'
        ])

        # If it is a greylist verb sentence, but there is no performance content such as numbers, units, time, etc. → delete
        if contains_graylist and not (has_numbers or has_units or has_percentage or has_time_ref or has_sustainability_context):
            logging.debug(f"KPI rejected (graylist verb, no quantitative data): {kpi_text[:100]}...")
            continue

        # More lenient quality scoring - only require numbers and either units/percentage OR time reference OR sustainability context
        basic_requirements = has_numbers and (has_percentage or has_units or has_time_ref or has_sustainability_context)

        # Additional check for obvious ESG relevance
        is_esg_relevant = any(esg_word in kpi_text for esg_word in [
            'emission', 'carbon', 'energy', 'waste', 'water', 'renewable', 'employee',
            'safety', 'training', 'diversity', 'governance', 'board', 'compliance',
            'sustainability', 'environmental', 'social', 'ghg', 'co2', 'workforce',
            'gender', 'health', 'injury', 'incident', 'ethics', 'transparency'
        ])

        if basic_requirements or is_esg_relevant:
            quality_kpis.append(kpi)
            logging.debug(f"KPI accepted: {kpi_text[:100]}...")
        else:
            logging.debug(f"KPI filtered out for quality: {kpi_text[:100]}...")

    logging.info(f"Quality validation: {len(quality_kpis)}/{len(kpis)} KPIs passed")
    return quality_kpis

In [13]:
# ============ Image processing functions ============
def extract_numeric_spans(page):
    text_dict = page.get_text("dict")
    nums = []
    for block in text_dict["blocks"]:
        for line in block.get("lines", []):
            for span in line.get("spans", []):
                s = span["text"].strip()
                if re.match(r"[\d,.]+%?$", s):          # Pure number or number + %
                    nums.append({
                        "text": s,
                        "bbox": span["bbox"],           # (x0,y0,x1,y1)
                        "font": span["size"]
                    })
    return nums

def extract_images_from_pdf_fixed(pdf_path: str) -> List[Dict]:
    """Extract images from PDF using PyMuPDF"""
    images = []

    try:
        pdf_document = fitz.open(pdf_path)

        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            image_list = page.get_images()

            # 🔥 New: Extract page screenshots as an alternative
            page_pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # high resolution
            page_img = Image.frombytes("RGB", [page_pix.width, page_pix.height], page_pix.samples)

            # Add full page screenshot
            images.append({
                'image': page_img,
                'page_number': page_num + 1,
                'width': page_img.width,
                'height': page_img.height,
                'image_index': 'full_page',
                'type': 'full_page'
            })


            for img_index, img in enumerate(image_list):
                try:
                    xref = img[0]
                    base_image = pdf_document.extract_image(xref)
                    image_bytes = base_image["image"]

                    image = Image.open(BytesIO(image_bytes))

                    # Convert to RGB if needed
                    if image.mode in ['RGBA', 'LA']:
                        background = Image.new('RGB', image.size, (255, 255, 255))
                        if image.mode == 'RGBA':
                            background.paste(image, mask=image.split()[-1])
                        else:
                            background.paste(image)
                        image = background
                    elif image.mode != 'RGB':
                        image = image.convert('RGB')

                    # Filter small images
                    if image.width >= 50 and image.height >= 50:
                        images.append({
                            'image': image,
                            'page_number': page_num + 1,
                            'width': image.width,
                            'height': image.height,
                            'image_index': img_index,
                            'type': 'extracted'  # 🔥 Added type identifier
                        })

                except Exception as e:
                    logging.warning(f"Error extracting image {img_index} from page {page_num + 1}: {e}")
                    continue

        pdf_document.close()
        logging.info(f"Extracted {len(images)} images from PDF")
        return images

    except Exception as e:
        logging.error(f"Error extracting images from PDF: {e}")
        return []

def image_to_base64_fixed(image: Image.Image) -> str:
    """Convert image to base64 with error handling"""
    try:
        if image.mode not in ['RGB', 'L']:
            image = image.convert('RGB')

        # Resize large images
        max_size = (1536, 1536)
        if image.width > max_size[0] or image.height > max_size[1]:
            # Calculate scaling to maintain aspect ratio
            ratio = min(max_size[0]/image.width, max_size[1]/image.height)
            new_size = (int(image.width * ratio), int(image.height * ratio))
            image = image.resize(new_size, Image.Resampling.LANCZOS)

        buffered = BytesIO()
        image.save(buffered, format="JPEG", quality=95)
        img_str = base64.b64encode(buffered.getvalue()).decode()

        return img_str

    except Exception as e:
        logging.error(f"Error converting image to base64: {e}")
        return ""

In [14]:
# ------------------------------------------------------------
# Multi-crop / multi-resolution generator (supports crop parameter 0)
# ------------------------------------------------------------
from itertools import product

def generate_image_variants(img: Image.Image,
                            max_side_full: int = 1200,
                            crop_size: int = 768,
                            stride: int = 512) -> List[Tuple[Image.Image, str]]:
    """
    Returns [(variant_image, variant_tag), ...]
    variant_tag value: original / resized / crop_{row}_{col}
    """
    variants = []

    # 0) Original image
    variants.append((img, "original"))

    # 1) Zoom (if the original image is too large)
    w, h = img.size
    if max(w, h) > max_side_full:
        scale = max_side_full / float(max(w, h))
        resized = img.resize((int(w * scale), int(h * scale)), Image.Resampling.LANCZOS)
        variants.append((resized, "resized"))
    else:
        resized = img  # Keep the original image without scaling
        variants.append((resized, "resized"))  # Unified plus resized version

    # 2) Sliding window cropping (skipped when cropping size or step size is 0)
    if crop_size > 0 and stride > 0:
        base_img = variants[-1][0]
        bw, bh = base_img.size
        if bw > crop_size or bh > crop_size:
            xs = list(range(0, max(bw - crop_size, 1), stride)) + [bw - crop_size]
            ys = list(range(0, max(bh - crop_size, 1), stride)) + [bh - crop_size]
            for r, c in product(range(len(ys)), range(len(xs))):
                x, y = xs[c], ys[r]
                crop = base_img.crop((x, y, x + crop_size, y + crop_size))
                # Filter solid color areas
                if np.array(crop.convert('L')).std() < 5:
                    continue
                variants.append((crop, f"crop_{r}_{c}"))

    return variants

In [15]:
# ---------------------------------------------
# 📊 A chart recognition function that replaces plotclassifier (Hugging Face model)
# ---------------------------------------------
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
# 🔧 Fix: Chart recognition with CLIP model
def setup_chart_classifier():
    """Setting up the chart classifier"""
    try:
        from transformers import CLIPProcessor, CLIPModel

        # Loading CLIP Model
        model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

        def is_chart_image_clip(image: Image.Image) -> bool:
            """Use CLIP to determine whether it is a chart"""
            try:
                # Defines text description related to the chart
                chart_labels = [
                    "a chart", "a graph", "a bar chart", "a pie chart",
                    "a line graph", "a table", "data visualization",
                    "statistics", "a diagram", "an infographic"
                ]

                # Processing Input
                inputs = processor(
                    text=chart_labels,
                    images=image,
                    return_tensors="pt",
                    padding=True
                )

                # Get prediction results
                outputs = model(**inputs)
                logits_per_image = outputs.logits_per_image
                probs = logits_per_image.softmax(dim=1)

                # If the probability of any chart label is greater than 0.25, it is considered to be a chart
                max_prob = probs.max().item()
                is_chart = max_prob > 0.25

                logging.debug(f"CLIP chart recognition: maximum probability = {max_prob:.3f}, result = {is_chart}")
                return is_chart

            except Exception as e:
                logging.warning(f"CLIP chart recognition failed: {e}")
                # Downgrade to statistical methods
                gray = image.convert('L')
                return np.array(gray).std() > 15

        logging.info("✅ Graph recognition using CLIP model")
        return is_chart_image_clip

    except ImportError:
        logging.warning("CLIP model is not available, use statistical methods")
        def is_chart_image_stats(image: Image.Image) -> bool:
            """Statistical method to determine whether it is a chart"""
            try:
                gray = image.convert('L')
                std_dev = np.array(gray).std()
                return std_dev > 15
            except:
                return True

        return is_chart_image_stats
    except Exception as e:
        logging.error(f"Failed to set chart classifier: {e}")
        def is_chart_image_fallback(image: Image.Image) -> bool:
            return True  # Conservative Strategy: When in Doubt, Analyze
        return is_chart_image_fallback

# Initialize the graph classifier
is_chart_image = setup_chart_classifier()


def extract_kpi_from_image_fixed(image: Image.Image, page_number: int, image_type: str = 'extracted') -> List[Dict]:
    """Extract KPIs from image with improved error handling"""
    try:
        # 🔥 New: Pre-filter: Check if it might be a chart
        if not is_chart_image(image):
            logging.debug(f"Image on page {page_number} filtered out (not likely a chart)")
            return []

        base64_image = image_to_base64_fixed(image)
        if not base64_image:
            return []

        # 🔥 Change: Use enhanced prompt
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": ENHANCED_IMAGE_KPI_SYSTEM_PROMPT  # 🔥 Using the new prompt
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            # 🔥 New: Detailed user instructions
                            "text": """Analyze this image carefully for quantifiable performance data.

IMPORTANT ANALYSIS PRINCIPLES:

1. **Chart Type Recognition**:
   - Stacked charts: Multiple colors/patterns layered in same position
   - Grouped charts: Multiple elements side by side at same position
   - Simple charts: One data point per position

2. **Value Extraction Rules**:
   - For STACKED charts: Read each layer separately, NOT the total height
   - For GROUPED charts: Read each element individually
   - For SIMPLE charts: Read data point values directly

3. **Data Relevance Filter**:
   ✅ EXTRACT: Performance outcomes, efficiency metrics, reduction rates, satisfaction scores, compliance rates
   ❌ SKIP: Certification counts, project timelines, implementation schedules, organizational charts, process flows

4. **Quality Standards**:
   - Only extract clear, quantifiable performance indicators
   - Each data point must have complete context
   - If uncertain about values, don't estimate
   - If chart shows mainly operational/administrative data, return empty array

Please analyze this chart step by step:
- First identify the chart type
- Then determine if it contains performance KPIs
- Finally extract all relevant performance data points

Focus on measurable outcomes and achievements, not counts or processes."""
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            temperature=0.1,
            max_tokens=4000,
            timeout=60
        )

        content = response.choices[0].message.content.strip()

        if not content:
            return []

        # Clean formatting
        if content.startswith('```json'):
            content = content[7:]
        if content.endswith('```'):
            content = content[:-3]

        content = content.strip()

        if not content.startswith("["):
            logging.warning(f"Image analysis response not JSON list: {content[:100]}...")
            return []

        try:
            result = json.loads(content)
        except json.JSONDecodeError as e:
            logging.warning(f"JSON parsing failed for image analysis: {e}")
            return []

        if not isinstance(result, list):
            return []

        # Process results
        processed_result = []
        for item in result:
            if isinstance(item, dict) and 'kpi_text' in item:
                if not item.get('kpi_text', '').strip():
                    continue

                item['source_page'] = page_number
                item['source_type'] = 'image'
                item['image_type'] = image_type  # 🔥 新增字段

                # 🔥 更改：确保有chart标识
                kpi_text = item['kpi_text']
                if not any(marker in kpi_text.lower() for marker in ['chart', 'graph', 'table', 'figure']):
                    chart_type = item.get('chart_type', 'chart')
                    item['kpi_text'] = f"[{chart_type.title()}] {kpi_text}"

                processed_result.append(item)

        if processed_result:
            logging.info(f"✅ Extracted {len(processed_result)} KPIs from {image_type} on page {page_number}")
        else:
            logging.debug(f"❌ No KPIs found in {image_type} on page {page_number}")

        return processed_result

    except Exception as e:
        logging.error(f"Error extracting KPIs from image: {e}")
        return []


def process_pdf_images_for_kpis_fixed(pdf_path: str) -> List[Dict]:
    """
    Traverse each page of the PDF:
        • Perform multiple cropping + Vision on all ‘extracted’ images on the page
        • If the page has not captured the KPI, perform Vision on the entire page screenshot
    """
    logging.info("Starting page-by-page image KPI extraction …")

    images = extract_images_from_pdf_fixed(pdf_path)
    if not images:
        return []

    # Aggregate images by page
    page_dict = {}
    for info in images:
        pg = info["page_number"]
        page_dict.setdefault(pg, {"extracted": [], "full": None})
        if info["type"] == "extracted":
            page_dict[pg]["extracted"].append(info["image"])
        else:                    # full_page
            page_dict[pg]["full"] = info["image"]

    all_image_kpis: List[Dict] = []

    # —— Page by page processing ——
    for pg in sorted(page_dict.keys()):
        logging.info(f"\n=== Page {pg} ===")
        page_kpis: List[Dict] = []

        # ① Individually extracted images
        for idx, img in enumerate(page_dict[pg]["extracted"]):
            for var_img, var_tag in generate_image_variants(img, 1200, 768, 512):
                kpis = extract_kpi_from_image_fixed(
                    var_img, pg, f"extracted_{var_tag}"
                )
                for k in kpis:
                    key = generate_universal_metric_key(k)
                    if key not in {generate_universal_metric_key(x) for x in page_kpis}:
                        page_kpis.append(k)
                time.sleep(0.8)

        # ② If it is still empty, analyze the entire page again
        if not page_kpis and page_dict[pg]["full"] is not None:
            for var_img, var_tag in generate_image_variants(
                    page_dict[pg]["full"], 1200, 0, 0):   # 只做 original/resized
                kpis = extract_kpi_from_image_fixed(
                    var_img, pg, f"full_{var_tag}"
                )
                for k in kpis:
                    key = generate_universal_metric_key(k)
                    if key not in {generate_universal_metric_key(x) for x in page_kpis}:
                        page_kpis.append(k)
                time.sleep(1.0)

        logging.info(f"  → Page {pg} KPI count: {len(page_kpis)}")
        all_image_kpis.extend(page_kpis)

    logging.info(f"Image KPI extraction finished: {len(all_image_kpis)} KPIs from {len(page_dict)} pages")
    return all_image_kpis

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [16]:
# 1. Conservative image filter
def conservative_image_filter(image: Image.Image) -> Tuple[bool, str]:
    """Conservative image filtering - only filters obviously useless images"""
    try:
        # Only filter very small images (maybe logos, icons)
        if image.width < 40 or image.height < 40:
            return False, "too_small_icon"

        # Filter only images of almost pure colors (decorative elements)
        gray = np.array(image.convert('L'))
        std_dev = gray.std()

        # Very conservative threshold - only images with completely pure colors are filtered
        if std_dev < 3:
            return False, "pure_color"

        # Check if it is a pure white background (blank area)
        mean_val = gray.mean()
        if mean_val > 250 and std_dev < 8:
            return False, "blank_white"

        # Default: Process all other images to ensure integrity
        return True, "keep_for_analysis"
    except Exception:
        return True, "filter_error_keep"

# 2. Cache mechanism
class FastKPICache:
    def __init__(self, cache_dir: str = "fast_kpi_cache"):
        self.cache_dir = cache_dir
        os.makedirs(cache_dir, exist_ok=True)
        self.hit_count = 0
        self.miss_count = 0

    def get_image_hash(self, image: Image.Image) -> str:
        """Fast image fingerprint generation"""
        width, height = image.size
        if width > 100 and height > 100:
            center_crop = image.crop((
                width//4, height//4,
                3*width//4, 3*height//4
            )).resize((32, 32))
            img_bytes = BytesIO()
            center_crop.save(img_bytes, format='JPEG', quality=50)
            sample_hash = hashlib.md5(img_bytes.getvalue()).hexdigest()[:16]
        else:
            sample_hash = hashlib.md5(str(width * height).encode()).hexdigest()[:16]

        return f"{width}x{height}_{sample_hash}"

    def get_cached_kpis(self, image_hash: str) -> Optional[List[Dict]]:
        cache_file = os.path.join(self.cache_dir, f"{image_hash}.pkl")
        if os.path.exists(cache_file):
            try:
                with open(cache_file, 'rb') as f:
                    self.hit_count += 1
                    return pickle.load(f)
            except:
                pass
        self.miss_count += 1
        return None

    def cache_kpis(self, image_hash: str, kpis: List[Dict]):
        cache_file = os.path.join(self.cache_dir, f"{image_hash}.pkl")
        try:
            with open(cache_file, 'wb') as f:
                pickle.dump(kpis, f)
        except:
            pass

    def get_stats(self):
        total = self.hit_count + self.miss_count
        hit_rate = self.hit_count / total if total > 0 else 0
        return f"Cache: {self.hit_count} hits, {self.miss_count} misses (hit rate: {hit_rate:.1%})"

# Initialize the cache
fast_cache = FastKPICache()

# 3. Optimized API calls
COMPREHENSIVE_EXTRACTION_PROMPT = """
You are an expert data analyst. Extract ALL quantifiable performance indicators from this image.

CRITICAL REQUIREMENTS:
1. Extract EVERY visible number, percentage, and metric
2. Include ALL data points from charts, graphs, and tables
3. Do not skip any quantifiable information

Return complete JSON array:
[
  {
    "kpi_text": "Complete contextual description with the specific number",
    "quantitative_value": "exact number only",
    "unit": "unit of measurement",
    "kpi_theme": "Environmental/Social/Governance",
    "kpi_category": "specific category",
    "time_period": "year/period if visible"
  }
]

COMPLETENESS IS CRITICAL - Extract everything quantifiable.
"""

def extract_kpi_optimized(image: Image.Image, page_number: int) -> List[Dict]:
    """Optimized KPI extraction"""
    try:
        # Check the cache
        image_hash = fast_cache.get_image_hash(image)
        cached_kpis = fast_cache.get_cached_kpis(image_hash)
        if cached_kpis is not None:
            for kpi in cached_kpis:
                kpi['source_page'] = page_number
            return cached_kpis

        # Optimizing image encoding
        base64_image = image_to_base64_optimized(image)
        if not base64_image:
            return []

        # API Calls
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "user",
                "content": [
                    {"type": "text", "text": COMPREHENSIVE_EXTRACTION_PROMPT},
                    {"type": "image_url", "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                        "detail": "high"
                    }}
                ]
            }],
            temperature=0.0,
            max_tokens=2000,
            timeout=60
        )

        # Parsing results
        kpis = parse_optimized_response(response, page_number)

        # Caching results
        fast_cache.cache_kpis(image_hash, kpis)

        return kpis

    except Exception as e:
        logging.warning(f"Optimized KPI extraction failed for page {page_number}: {e}")
        return []

def image_to_base64_optimized(image: Image.Image) -> str:
    """Optimized image encoding"""
    try:
        max_dimension = 1400  # Maintain high quality
        width, height = image.size

        if max(width, height) > max_dimension:
            scale = max_dimension / max(width, height)
            new_size = (int(width * scale), int(height * scale))
            image = image.resize(new_size, Image.Resampling.LANCZOS)

        if image.mode != 'RGB':
            if image.mode in ['RGBA', 'LA']:
                background = Image.new('RGB', image.size, (255, 255, 255))
                if image.mode == 'RGBA':
                    background.paste(image, mask=image.split()[-1])
                else:
                    background.paste(image)
                image = background
            else:
                image = image.convert('RGB')

        buffered = BytesIO()
        image.save(buffered, format="JPEG", quality=92, optimize=True)
        return base64.b64encode(buffered.getvalue()).decode()

    except Exception as e:
        logging.error(f"Optimized image encoding failed: {e}")
        return ""

def parse_optimized_response(response, page_number: int) -> List[Dict]:
    """Optimized response parsing"""
    try:
        content = response.choices[0].message.content.strip()

        if content.startswith('```json'):
            content = content[7:]
        if content.endswith('```'):
            content = content[:-3]
        content = content.strip()

        if not content.startswith('['):
            return []

        result = json.loads(content)
        if not isinstance(result, list):
            return []

        validated_kpis = []
        for item in result:
            if (isinstance(item, dict) and
                item.get('kpi_text', '').strip() and
                item.get('quantitative_value', '').strip()):

                kpi = {
                    'kpi_text': item.get('kpi_text', '').strip(),
                    'quantitative_value': str(item.get('quantitative_value', '')).strip(),
                    'unit': item.get('unit', '').strip(),
                    'kpi_theme': item.get('kpi_theme', 'Environmental').strip(),
                    'kpi_category': item.get('kpi_category', '').strip(),
                    'time_period': item.get('time_period', '').strip(),
                    'source_page': page_number,
                    'source_type': 'image'
                }
                validated_kpis.append(kpi)

        return validated_kpis

    except Exception as e:
        logging.warning(f"Optimized response parsing failed: {e}")
        return []

# 4. Parallel image processing
def process_images_in_parallel(image_data: List[Dict], max_workers: int = 3) -> List[Dict]:
    """Parallel image processing"""
    if not image_data:
        return []

    print(f"🔄 Processing {len(image_data)} images in parallel...")

    all_kpis = []
    completed_count = 0

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_info = {}
        for img_info in image_data:
            future = executor.submit(
                extract_kpi_optimized,
                img_info['image'],
                img_info['page_number']
            )
            future_to_info[future] = img_info

        for future in concurrent.futures.as_completed(future_to_info):
            img_info = future_to_info[future]
            try:
                kpis = future.result(timeout=90)
                all_kpis.extend(kpis)
                completed_count += 1

                if completed_count % 5 == 0:
                    progress = completed_count / len(image_data) * 100
                    print(f"   📈 Progress: {completed_count}/{len(image_data)} ({progress:.1f}%)")

            except Exception as e:
                logging.warning(f"Image processing failed for page {img_info['page_number']}: {e}")
                continue

    print(f"📊 Parallel processing completed: {len(all_kpis)} KPIs extracted")
    print(f"📋 {fast_cache.get_stats()}")

    return all_kpis

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# ============ Main processing function ============
def process_sustainability_report_with_enhanced_images(pdf_path: str) -> pd.DataFrame:
    """Main processing function with image analysis"""
    logging.info("Starting enhanced PDF processing with image analysis...")

    # Step 1: Text and table extraction
    logging.info("Step 1/5: Reading PDF text and tables...")
    full_text = pdf_to_text_and_tables(pdf_path)

    camelot_tables = camelot_extra_tables_enhanced(pdf_path)
    if camelot_tables:
        full_text += "\n\n" + "\n\n".join(camelot_tables)

    logging.info("Step 2/5: Chunking text...")
    chunks = split_into_chunks(full_text, MAX_TOKENS_CHUNK)

    logging.info("Step 3/5: Extracting KPIs from text...")
    text_kpis = []
    for idx, chunk in enumerate(chunks, 1):
        logging.info(f"Processing text chunk {idx}/{len(chunks)}")
        if chunk.strip():
            chunk_kpis = extract_kpi_from_chunk_universal(chunk)
            text_kpis.extend(chunk_kpis)
            if idx < len(chunks):
                time.sleep(SLEEP_SEC)

    # Step 4: Image KPI extraction
    logging.info("Step 4/5: Extracting KPIs from images...")
    image_kpis = process_pdf_images_for_kpis_fixed(pdf_path)

    # Step 5: Combine and process
    logging.info("Step 5/5: Combining and processing all KPIs...")

    for kpi in text_kpis:
        if 'source_type' not in kpi:
            kpi['source_type'] = 'text'

    all_kpis = text_kpis + image_kpis
    all_kpis = post_process_kpis_universal(all_kpis)

    df_auto = pd.DataFrame(all_kpis)

    if not df_auto.empty:
        if 'source_type' not in df_auto.columns:
            df_auto['source_type'] = 'text'

        initial_count = len(df_auto)
        df_auto = df_auto.drop_duplicates(subset=['kpi_text'], keep='first')
        final_count = len(df_auto)

        logging.info(f"Removed {initial_count - final_count} duplicate KPIs")

        try:
            df_auto = df_auto.sort_values(['source_type', 'kpi_theme', 'kpi_category'], na_position='last')
        except KeyError:
            pass

        text_kpi_count = len([kpi for kpi in all_kpis if kpi.get('source_type', 'text') != 'image'])
        image_kpi_count = len([kpi for kpi in all_kpis if kpi.get('source_type') == 'image'])

        logging.info(f"KPI Summary: {text_kpi_count} from text/tables, {image_kpi_count} from images")

    return df_auto


In [19]:
# ============ Optimize moderator processing function ============
def process_sustainability_report_OPTIMIZED(pdf_path: str) -> pd.DataFrame:
    """Optimize moderator processing functions - improve performance while ensuring integrity"""

    start_time = time.time()
    print("⚡ Starting OPTIMIZED processing with completeness guarantee...")

    try:
        # Parallel text and image preprocessing
        with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
            print("🔄 Starting parallel text and image preprocessing...")

            # Text processing (using your existing logic)
            def extract_text_kpis():
                full_text = pdf_to_text_and_tables(pdf_path)
                camelot_tables = camelot_extra_tables_enhanced(pdf_path)
                if camelot_tables:
                    full_text += "\n\n" + "\n\n".join(camelot_tables)

                chunks = split_into_chunks(full_text, MAX_TOKENS_CHUNK)
                text_kpis = []
                for idx, chunk in enumerate(chunks, 1):
                    if chunk.strip():
                        chunk_kpis = extract_kpi_from_chunk_universal(chunk)
                        text_kpis.extend(chunk_kpis)
                        if idx < len(chunks):
                            time.sleep(SLEEP_SEC)
                return text_kpis

            #Image preprocessing (using optimized filtering)
            def extract_and_filter_images():
                all_images = extract_images_from_pdf_fixed(pdf_path)
                filtered_images = []

                for img_info in all_images:
                    should_process, reason = conservative_image_filter(img_info['image'])
                    if should_process:
                        filtered_images.append(img_info)

                print(f"📊 Conservative filtering: Kept {len(filtered_images)}/{len(all_images)} images")
                return filtered_images

            text_future = executor.submit(extract_text_kpis)
            image_future = executor.submit(extract_and_filter_images)

            text_kpis = text_future.result()
            image_data = image_future.result()

        preprocessing_time = time.time() - start_time
        print(f"⏱️  Preprocessing completed in {preprocessing_time:.1f}s")

        # Parallel Image KPI Extraction
        image_start = time.time()
        image_kpis = process_images_in_parallel(image_data, max_workers=3)
        image_time = time.time() - image_start
        print(f"⏱️  Image processing completed in {image_time:.1f}s")

        # Post-processing
        all_kpis = text_kpis + image_kpis
        all_kpis = post_process_kpis_universal(all_kpis)

        # Convert to DataFrame
        df_auto = pd.DataFrame(all_kpis)

        if not df_auto.empty:
            initial_count = len(df_auto)
            df_auto = df_auto.drop_duplicates(subset=['kpi_text'], keep='first')
            final_count = len(df_auto)

            if 'source_type' not in df_auto.columns:
                df_auto['source_type'] = 'text'

            print(f"🔄 Removed {initial_count - final_count} exact duplicates")

        # Performance Statistics
        total_time = time.time() - start_time
        text_count = len([k for k in all_kpis if k.get('source_type') != 'image'])
        image_count = len([k for k in all_kpis if k.get('source_type') == 'image'])

        print(f"\n⚡ OPTIMIZED processing completed!")
        print(f"⏱️  Total time: {total_time:.1f}s ({total_time/60:.1f}min)")
        print(f"📊 Results:")
        print(f"   - Text/Tables: {text_count} KPIs")
        print(f"   - Images/Charts: {image_count} KPIs")
        print(f"   - Total unique: {len(df_auto)} KPIs")
        print(f"⚡ Performance: {len(df_auto)/total_time:.1f} KPIs/second")

        return df_auto

    except Exception as e:
        total_time = time.time() - start_time
        print(f"❌ Optimized processing failed after {total_time:.1f}s: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

In [20]:
# ============ Result saving and comparison functions ============
def infer_stakeholder(row) -> str:
    """Infer affected stakeholders based on KPI theme and category"""
    theme = row.get('kpi_theme', '').lower()
    category = row.get('kpi_category', '').lower()
    kpi_text = row.get('kpi_text', '').lower()

    if theme == 'environmental':
        return "Environment, Community, Future Generations"
    elif theme == 'social':
        if 'employee' in category or 'workforce' in category or 'gender' in category:
            return "Employees"
        elif 'customer' in category or 'safety' in category:
            return "Customers, Community"
        elif 'community' in category:
            return "Local Communities"
        elif 'supply' in category or 'supplier' in kpi_text:
            return "Suppliers, Business Partners"
        else:
            return "Employees, Community"
    elif theme == 'governance':
        if 'board' in category:
            return "Shareholders, Investors"
        elif 'cyber' in category or 'data' in category:
            return "Customers, Employees, Business Partners"
        else:
            return "Shareholders, Investors, Stakeholders"
    else:
        return "All Stakeholders"

def save_results(df_auto: pd.DataFrame, output_path: str, pdf_path: str = "") -> None:
    """Save results to Excel file with proper formatting"""
    try:
        os.makedirs(os.path.dirname(output_path) if os.path.dirname(output_path) else '.', exist_ok=True)

        if not df_auto.empty:
            # Add metadata columns
            pdf_filename = os.path.basename(pdf_path) if pdf_path else "Unknown"
            df_auto['PDF file name'] = pdf_filename
            df_auto['Title of the report'] = ""

            if 'source_page' in df_auto.columns:
                df_auto['Absolute Page Number'] = df_auto['source_page']
                df_auto = df_auto.drop('source_page', axis=1)
            else:
                df_auto['Absolute Page Number'] = "Unknown"

            df_auto['Impacted Stakeholder'] = df_auto.apply(infer_stakeholder, axis=1)

            # Reorder columns
            original_columns = [col for col in df_auto.columns if col not in
                              ['PDF file name', 'Title of the report', 'Absolute Page Number', 'Impacted Stakeholder']]
            new_column_order = ['PDF file name', 'Title of the report', 'Absolute Page Number', 'Impacted Stakeholder'] + original_columns
            df_auto = df_auto[new_column_order]

        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            df_auto.to_excel(writer, sheet_name='Auto_KPIs', index=False)

            if not df_auto.empty:
                # Theme summary
                theme_summary = df_auto.groupby('kpi_theme').size().reset_index(name='count')
                theme_summary.to_excel(writer, sheet_name='Theme_Summary', index=False)

                # Category summary
                category_summary = df_auto.groupby(['kpi_theme', 'kpi_category']).size().reset_index(name='count')
                category_summary.to_excel(writer, sheet_name='Category_Summary', index=False)

        logging.info(f"Results saved to {output_path}")

    except Exception as e:
        logging.error(f"Error saving results: {e}")

In [21]:
# ============ Main execution function ============
def main():
    """Enhanced main execution function with validation"""
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s: %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S"
    )

    try:
        if not os.path.exists(PDF_PATH):
            logging.error(f"PDF file not found: {PDF_PATH}")
            return

        # Process the PDF
        df_auto = process_sustainability_report_with_enhanced_images(PDF_PATH)

        # Save results
        save_results(df_auto, EXPORT_AUTO_XLSX, PDF_PATH)
        logging.info(f"KPI extraction completed: {len(df_auto)} KPIs extracted")

        # Enhanced validation with comprehensive analysis
        if MANUAL_XLSX and Path(MANUAL_XLSX).exists():
            print("\n🔍 Running comprehensive validation...")
            validation_results = enhanced_compare_with_manual_kpis(
                df_auto, MANUAL_XLSX, "comprehensive_validation"
            )

            if validation_results:
                print("✅ Validation completed with detailed analysis!")
                print(f"📁 Detailed results saved to: comprehensive_validation/")
            else:
                print("⚠️ Validation encountered issues")
        else:
            logging.info("Manual KPI file not found, skipping validation.")

        # Display summary
        if not df_auto.empty:
            print(f"\n=== Extraction Summary ===")
            print(f"Total KPIs extracted: {len(df_auto)}")

            # Source statistics
            if 'source_type' in df_auto.columns:
                source_counts = df_auto['source_type'].value_counts()
                print(f"From text/tables: {source_counts.get('text', 0)}")
                print(f"From images/charts: {source_counts.get('image', 0)}")

            # Theme statistics
            if 'kpi_theme' in df_auto.columns:
                theme_counts = df_auto['kpi_theme'].value_counts()
                print(f"\nKPI Distribution by Theme:")
                for theme, count in theme_counts.items():
                    print(f"  {theme}: {count}")
        else:
            print("\nNo KPIs were extracted from the document.")

    except Exception as e:
        logging.error(f"Error in main execution: {e}")
        import traceback
        traceback.print_exc()

In [22]:
# ============ Auxiliary functions ============
def install_dependencies():
    """Install required dependencies"""
    try:
        import subprocess
        import sys

        dependencies = [
            "openai",
            "python-dotenv",
            "pdfplumber",
            "tiktoken",
            "pandas",
            "PyMuPDF",
            "Pillow",
            "openpyxl"
        ]

        for dep in dependencies:
            try:
                __import__(dep.replace('-', '_'))
                print(f"✅ {dep} is already installed")
            except ImportError:
                print(f"Installing {dep}...")
                subprocess.check_call([sys.executable, "-m", "pip", "install", dep])
                print(f"✅ Installed {dep}")

        # Optional Camelot installation
        try:
            import camelot
            print("✅ Camelot is already installed")
        except ImportError:
            print("Installing Camelot (optional)...")
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "camelot-py[cv]"])
                print("✅ Installed Camelot")
            except:
                print("⚠️ Camelot installation failed (optional dependency)")

        print("🎉 All dependencies checked/installed successfully!")

    except Exception as e:
        print(f"❌ Error with dependencies: {e}")

def validate_environment():
    """Validate environment setup"""
    issues = []

    # Check API key
    if not os.getenv("OPENAI_API_KEY"):
        issues.append("OPENAI_API_KEY not found in environment variables")

    # Check PDF file
    if not os.path.exists(PDF_PATH):
        issues.append(f"PDF file not found: {PDF_PATH}")

    # Check required imports
    required_modules = ['openai', 'pdfplumber', 'pandas', 'tiktoken', 'PIL', 'fitz']
    for module in required_modules:
        try:
            __import__(module)
        except ImportError:
            issues.append(f"Required module '{module}' not installed")

    if issues:
        print("❌ Environment validation failed:")
        for issue in issues:
            print(f"  - {issue}")
        return False
    else:
        print("✅ Environment validation passed")
        return True


In [23]:
# ============ Simplified execution interface ============
def run_kpi_extraction():
    """Simplified interface to run KPI extraction"""
    print("🚀 Starting KPI extraction process...")

    # Validate environment
    if not validate_environment():
        print("Please fix the environment issues before running.")
        return

    # Run main function
    main()

In [24]:
# ============ Optimized execution interface ============
def run_optimized_kpi_extraction():
    """Run optimized KPI extraction"""
    print("⚡ Starting OPTIMIZED KPI extraction...")
    print("🎯 Goal: Extract ALL KPIs with 60-70% better performance")

    # Verify the environment
    if not validate_environment():
        print("Please fix the environment issues before running.")
        return None

    try:
        # Run optimization process
        df_results = process_sustainability_report_OPTIMIZED(PDF_PATH)

        # Save the results
        output_file = "OPTIMIZED_" + EXPORT_AUTO_XLSX
        save_results(df_results, output_file, PDF_PATH)
        print(f"💾 Results saved to: {output_file}")

        # Show Statistics
        if not df_results.empty and 'source_type' in df_results.columns:
            source_counts = df_results['source_type'].value_counts()
            print(f"\n📈 Final Statistics:")
            for source, count in source_counts.items():
                print(f"   - {source}: {count} KPIs")

        return df_results

    except Exception as e:
        print(f"❌ Optimized extraction failed: {e}")
        return None

def compare_original_vs_optimized():
    """Compare the performance of the original version and the optimized version"""
    print("🔬 Performance Comparison Test")
    print("=" * 50)

    # Test the original version
    print("\n📊 Testing Original Version...")
    original_start = time.time()
    try:
        original_df = process_sustainability_report_with_enhanced_images(PDF_PATH)
        original_time = time.time() - original_start
        print(f"⏱️  Original version: {original_time:.1f}s, {len(original_df)} KPIs")
    except Exception as e:
        print(f"❌ Original version failed: {e}")
        original_time = 999
        original_df = pd.DataFrame()

    # Test optimized version
    print("\n⚡ Testing Optimized Version...")
    optimized_start = time.time()
    try:
        optimized_df = process_sustainability_report_OPTIMIZED(PDF_PATH)
        optimized_time = time.time() - optimized_start
        print(f"⏱️  Optimized version: {optimized_time:.1f}s, {len(optimized_df)} KPIs")
    except Exception as e:
        print(f"❌ Optimized version failed: {e}")
        optimized_time = 999
        optimized_df = pd.DataFrame()

    # Performance comparison
    if original_time < 999 and optimized_time < 999:
        speedup = original_time / optimized_time
        time_saved = original_time - optimized_time
        kpi_diff = abs(len(optimized_df) - len(original_df))

        print(f"\n🚀 Performance Results:")
        print(f"   - Speed improvement: {speedup:.1f}x faster")
        print(f"   - Time saved: {time_saved:.1f}s ({time_saved/60:.1f}min)")
        print(f"   - KPI difference: {kpi_diff} KPIs")
        print(f"   - Completeness: {len(optimized_df)/len(original_df)*100:.1f}% of original" if len(original_df) > 0 else "")

        return {"original": original_df, "optimized": optimized_df, "speedup": speedup}

    return None

In [25]:
class KPIValidationPipeline:
    """Comprehensive KPI validation and evaluation system"""

    def __init__(self, manual_excel_path: str, auto_excel_path: str,
                 output_dir: str = "validation_results"):
        """
        Initialize validation pipeline

        Args:
            manual_excel_path: Path to manual KPI annotations
            auto_excel_path: Path to automatically extracted KPIs
            output_dir: Directory to save validation results
        """
        self.manual_path = manual_excel_path
        self.auto_path = auto_excel_path
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)

        # Load data
        self.manual_df = self._load_excel_safe(manual_excel_path, "manual")
        self.auto_df = self._load_excel_safe(auto_excel_path, "auto")

        # Validation results
        self.validation_results = {}
        self.detailed_analysis = {}

        # Similarity thresholds
        self.similarity_thresholds = {
            'exact': 1.0,
            'high': 0.9,
            'medium': 0.7,
            'low': 0.5
        }

        logging.info(f"Validation pipeline initialized:")
        logging.info(f"  Manual KPIs: {len(self.manual_df)}")
        logging.info(f"  Auto KPIs: {len(self.auto_df)}")

    def _load_excel_safe(self, filepath: str, source_type: str) -> pd.DataFrame:
        """Safely load Excel file with error handling"""
        try:
            if not Path(filepath).exists():
                logging.warning(f"{source_type.title()} file not found: {filepath}")
                return pd.DataFrame()

            df = pd.read_excel(filepath)
            logging.info(f"Loaded {source_type} file: {len(df)} rows")

            # Standardize column names
            df.columns = df.columns.str.strip().str.lower()

            # Ensure required columns exist
            required_cols = ['kpi_text']
            for col in required_cols:
                if col not in df.columns:
                    # Try to find similar column names
                    similar_cols = [c for c in df.columns if 'kpi' in c.lower() or 'text' in c.lower()]
                    if similar_cols:
                        df['kpi_text'] = df[similar_cols[0]]
                        logging.info(f"Using column '{similar_cols[0]}' as kpi_text")
                    else:
                        logging.warning(f"Required column '{col}' not found in {source_type} file")
                        df['kpi_text'] = ""

            # Clean text data
            df['kpi_text'] = df['kpi_text'].astype(str).str.strip()
            df = df[df['kpi_text'] != ''].reset_index(drop=True)

            return df

        except Exception as e:
            logging.error(f"Error loading {source_type} file: {e}")
            return pd.DataFrame()

    def normalize_text(self, text: str) -> str:
        """Normalize text for comparison"""
        if pd.isna(text) or text == '':
            return ""

        # Convert to string and lowercase
        text = str(text).lower().strip()

        # Remove extra whitespace
        text = ' '.join(text.split())

        # Remove common punctuation but keep percentages and numbers
        text = re.sub(r'[^\w\s\%\.\,\-]', ' ', text)

        # Normalize number formats
        text = re.sub(r'\b(\d+),(\d+)\b', r'\1\2', text)  # Remove commas in numbers
        text = re.sub(r'\s+', ' ', text)  # Normalize spaces

        return text.strip()

    def calculate_text_similarity(self, text1: str, text2: str) -> Dict[str, float]:
        """Calculate multiple similarity metrics between two texts"""
        norm1 = self.normalize_text(text1)
        norm2 = self.normalize_text(text2)

        if not norm1 or not norm2:
            return {'sequence': 0.0, 'cosine': 0.0, 'jaccard': 0.0, 'combined': 0.0}

        # 1. Sequence similarity (exact match)
        sequence_sim = SequenceMatcher(None, norm1, norm2).ratio()

        # 2. Cosine similarity (semantic)
        try:
            vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
            tfidf_matrix = vectorizer.fit_transform([norm1, norm2])
            cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        except:
            cosine_sim = 0.0

        # 3. Jaccard similarity (token overlap)
        tokens1 = set(norm1.split())
        tokens2 = set(norm2.split())
        if tokens1 or tokens2:
            jaccard_sim = len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))
        else:
            jaccard_sim = 0.0

        # 4. Combined similarity
        combined_sim = (sequence_sim * 0.4 + cosine_sim * 0.4 + jaccard_sim * 0.2)

        return {
            'sequence': sequence_sim,
            'cosine': cosine_sim,
            'jaccard': jaccard_sim,
            'combined': combined_sim
        }

    def find_matches(self, threshold: float = 0.7, similarity_type: str = 'combined') -> pd.DataFrame:
        """Find matches between manual and auto KPIs"""
        matches = []
        auto_matched = set()

        for manual_idx, manual_row in self.manual_df.iterrows():
            manual_text = manual_row['kpi_text']
            best_match = None
            best_similarity = 0.0

            for auto_idx, auto_row in self.auto_df.iterrows():
                if auto_idx in auto_matched:
                    continue

                auto_text = auto_row['kpi_text']
                similarities = self.calculate_text_similarity(manual_text, auto_text)
                similarity = similarities[similarity_type]

                if similarity > best_similarity and similarity >= threshold:
                    best_similarity = similarity
                    best_match = {
                        'manual_idx': manual_idx,
                        'auto_idx': auto_idx,
                        'manual_text': manual_text,
                        'auto_text': auto_text,
                        'similarity': similarity,
                        'all_similarities': similarities
                    }

            if best_match:
                matches.append(best_match)
                auto_matched.add(best_match['auto_idx'])

        return pd.DataFrame(matches)

    def calculate_metrics_at_threshold(self, threshold: float = 0.7,
                                     similarity_type: str = 'combined') -> Dict[str, float]:
        """Calculate precision, recall, F1 at specific threshold"""
        matches_df = self.find_matches(threshold, similarity_type)

        true_positives = len(matches_df)
        false_positives = len(self.auto_df) - true_positives
        false_negatives = len(self.manual_df) - true_positives

        # Calculate metrics
        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0.0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0.0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        return {
            'threshold': threshold,
            'similarity_type': similarity_type,
            'true_positives': true_positives,
            'false_positives': false_positives,
            'false_negatives': false_negatives,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'total_manual': len(self.manual_df),
            'total_auto': len(self.auto_df),
            'match_rate': true_positives / len(self.manual_df) if len(self.manual_df) > 0 else 0.0
        }

    def run_comprehensive_evaluation(self) -> Dict[str, any]:
        """Run comprehensive evaluation across multiple thresholds and similarity types"""
        logging.info("Running comprehensive evaluation...")

        results = {
            'threshold_analysis': [],
            'similarity_type_analysis': [],
            'category_analysis': {},
            'detailed_matches': {},
            'false_positives': [],
            'false_negatives': []
        }

        # 1. Threshold analysis
        thresholds = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        similarity_types = ['combined', 'sequence', 'cosine', 'jaccard']

        for threshold in thresholds:
            for sim_type in similarity_types:
                metrics = self.calculate_metrics_at_threshold(threshold, sim_type)
                results['threshold_analysis'].append(metrics)

        # 2. Find optimal threshold
        best_f1 = 0.0
        best_config = None
        for metrics in results['threshold_analysis']:
            if metrics['f1_score'] > best_f1:
                best_f1 = metrics['f1_score']
                best_config = (metrics['threshold'], metrics['similarity_type'])

        # 3. Detailed analysis at optimal threshold
        if best_config:
            optimal_threshold, optimal_sim_type = best_config
            logging.info(f"Optimal configuration: threshold={optimal_threshold}, similarity={optimal_sim_type}")

            matches_df = self.find_matches(optimal_threshold, optimal_sim_type)
            results['detailed_matches'] = matches_df.to_dict('records')

            # Find false positives and false negatives
            matched_auto_indices = set(matches_df['auto_idx'].tolist()) if not matches_df.empty else set()
            matched_manual_indices = set(matches_df['manual_idx'].tolist()) if not matches_df.empty else set()

            # False positives (auto KPIs not matched to manual)
            fp_indices = set(range(len(self.auto_df))) - matched_auto_indices
            results['false_positives'] = [
                {
                    'index': idx,
                    'kpi_text': self.auto_df.iloc[idx]['kpi_text'],
                    'category': self.auto_df.iloc[idx].get('kpi_category', 'Unknown'),
                    'theme': self.auto_df.iloc[idx].get('kpi_theme', 'Unknown'),
                    'source': self.auto_df.iloc[idx].get('source_type', 'Unknown')
                }
                for idx in fp_indices
            ]

            # False negatives (manual KPIs not matched by auto)
            fn_indices = set(range(len(self.manual_df))) - matched_manual_indices
            results['false_negatives'] = [
                {
                    'index': idx,
                    'kpi_text': self.manual_df.iloc[idx]['kpi_text'],
                    'category': self.manual_df.iloc[idx].get('kpi_category', 'Unknown'),
                    'theme': self.manual_df.iloc[idx].get('kpi_theme', 'Unknown')
                }
                for idx in fn_indices
            ]

        # 4. Category-level analysis
        if 'kpi_category' in self.manual_df.columns and 'kpi_category' in self.auto_df.columns:
            results['category_analysis'] = self._analyze_by_category()

        # 5. Theme-level analysis
        if 'kpi_theme' in self.manual_df.columns and 'kpi_theme' in self.auto_df.columns:
            results['theme_analysis'] = self._analyze_by_theme()

        self.validation_results = results
        return results

    def _analyze_by_category(self) -> Dict[str, Dict]:
        """Analyze performance by KPI category"""
        category_results = {}

        manual_categories = self.manual_df['kpi_category'].value_counts()
        auto_categories = self.auto_df['kpi_category'].value_counts()

        all_categories = set(manual_categories.index) | set(auto_categories.index)

        for category in all_categories:
            manual_count = manual_categories.get(category, 0)
            auto_count = auto_categories.get(category, 0)

            # Find matches within this category
            manual_cat_df = self.manual_df[self.manual_df['kpi_category'] == category]
            auto_cat_df = self.auto_df[self.auto_df['kpi_category'] == category]

            category_matches = 0
            if not manual_cat_df.empty and not auto_cat_df.empty:
                for _, manual_row in manual_cat_df.iterrows():
                    best_sim = 0.0
                    for _, auto_row in auto_cat_df.iterrows():
                        sim = self.calculate_text_similarity(
                            manual_row['kpi_text'],
                            auto_row['kpi_text']
                        )['combined']
                        best_sim = max(best_sim, sim)
                    if best_sim >= 0.7:
                        category_matches += 1

            category_precision = category_matches / auto_count if auto_count > 0 else 0.0
            category_recall = category_matches / manual_count if manual_count > 0 else 0.0
            category_f1 = 2 * (category_precision * category_recall) / (category_precision + category_recall) if (category_precision + category_recall) > 0 else 0.0

            category_results[category] = {
                'manual_count': manual_count,
                'auto_count': auto_count,
                'matches': category_matches,
                'precision': category_precision,
                'recall': category_recall,
                'f1_score': category_f1
            }

        return category_results

    def _analyze_by_theme(self) -> Dict[str, Dict]:
        """Analyze performance by KPI theme"""
        theme_results = {}

        manual_themes = self.manual_df['kpi_theme'].value_counts()
        auto_themes = self.auto_df['kpi_theme'].value_counts()

        all_themes = set(manual_themes.index) | set(auto_themes.index)

        for theme in all_themes:
            manual_count = manual_themes.get(theme, 0)
            auto_count = auto_themes.get(theme, 0)

            theme_results[theme] = {
                'manual_count': manual_count,
                'auto_count': auto_count,
                'coverage': auto_count / manual_count if manual_count > 0 else 0.0
            }

        return theme_results

    def generate_visualizations(self):
        """Generate comprehensive visualizations"""
        if not self.validation_results:
            logging.warning("No validation results found. Run evaluation first.")
            return

        # Set style
        try:
            plt.style.use('seaborn-v0_8')
        except:
            plt.style.use('seaborn')  # 备用样式
        fig = plt.figure(figsize=(20, 16))

        # 1. Threshold analysis
        threshold_df = pd.DataFrame(self.validation_results['threshold_analysis'])

        plt.subplot(3, 3, 1)
        for sim_type in threshold_df['similarity_type'].unique():
            data = threshold_df[threshold_df['similarity_type'] == sim_type]
            plt.plot(data['threshold'], data['f1_score'], marker='o', label=sim_type)
        plt.xlabel('Similarity Threshold')
        plt.ylabel('F1 Score')
        plt.title('F1 Score vs Threshold by Similarity Type')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # 2. Precision-Recall curve
        plt.subplot(3, 3, 2)
        for sim_type in threshold_df['similarity_type'].unique():
            data = threshold_df[threshold_df['similarity_type'] == sim_type]
            plt.plot(data['recall'], data['precision'], marker='o', label=sim_type)
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curves')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # 3. Category analysis
        if 'category_analysis' in self.validation_results:
            plt.subplot(3, 3, 3)
            cat_analysis = self.validation_results['category_analysis']
            categories = list(cat_analysis.keys())[:10]  # Top 10 categories
            f1_scores = [cat_analysis[cat]['f1_score'] for cat in categories]

            plt.barh(categories, f1_scores)
            plt.xlabel('F1 Score')
            plt.title('F1 Score by Category (Top 10)')
            plt.tight_layout()

        # 4. Theme distribution comparison
        plt.subplot(3, 3, 4)
        if 'kpi_theme' in self.manual_df.columns:
            manual_themes = self.manual_df['kpi_theme'].value_counts()
            auto_themes = self.auto_df['kpi_theme'].value_counts()

            x = np.arange(len(manual_themes))
            width = 0.35

            plt.bar(x - width/2, manual_themes.values, width, label='Manual', alpha=0.8)
            plt.bar(x + width/2, auto_themes.reindex(manual_themes.index, fill_value=0).values,
                   width, label='Auto', alpha=0.8)

            plt.xlabel('Theme')
            plt.ylabel('Count')
            plt.title('KPI Count by Theme')
            plt.xticks(x, manual_themes.index, rotation=45)
            plt.legend()

        # 5. Similarity distribution
        plt.subplot(3, 3, 5)
        if self.validation_results['detailed_matches']:
            similarities = [match['similarity'] for match in self.validation_results['detailed_matches']]
            plt.hist(similarities, bins=20, edgecolor='black', alpha=0.7)
            plt.xlabel('Similarity Score')
            plt.ylabel('Frequency')
            plt.title('Distribution of Similarity Scores (Matches)')
            plt.grid(True, alpha=0.3)

        # 6. Error analysis
        plt.subplot(3, 3, 6)
        fp_count = len(self.validation_results['false_positives'])
        fn_count = len(self.validation_results['false_negatives'])
        tp_count = len(self.validation_results['detailed_matches'])

        labels = ['True Positives', 'False Positives', 'False Negatives']
        counts = [tp_count, fp_count, fn_count]
        colors = ['green', 'red', 'orange']

        plt.pie(counts, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        plt.title('Classification Results')

        # 7. Coverage by source type
        plt.subplot(3, 3, 7)
        if 'source_type' in self.auto_df.columns:
            source_counts = self.auto_df['source_type'].value_counts()
            plt.pie(source_counts.values, labels=source_counts.index, autopct='%1.1f%%')
            plt.title('Auto KPIs by Source Type')

        # 8. Performance metrics summary
        plt.subplot(3, 3, 8)
        best_metrics = max(self.validation_results['threshold_analysis'],
                          key=lambda x: x['f1_score'])

        metrics = ['Precision', 'Recall', 'F1 Score']
        values = [best_metrics['precision'], best_metrics['recall'], best_metrics['f1_score']]

        bars = plt.bar(metrics, values, color=['skyblue', 'lightcoral', 'lightgreen'])
        plt.ylabel('Score')
        plt.title(f'Best Performance Metrics\n(Threshold: {best_metrics["threshold"]})')
        plt.ylim(0, 1)

        # Add value labels on bars
        for bar, value in zip(bars, values):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    f'{value:.3f}', ha='center', va='bottom')

        # 9. Match quality distribution
        plt.subplot(3, 3, 9)
        if self.validation_results['detailed_matches']:
            match_similarities = [match['similarity'] for match in self.validation_results['detailed_matches']]
            quality_bins = [0.5, 0.7, 0.8, 0.9, 1.0]
            quality_labels = ['Medium', 'Good', 'Very Good', 'Excellent']

            quality_counts = []
            for i in range(len(quality_bins)-1):
                count = sum(1 for sim in match_similarities
                          if quality_bins[i] <= sim < quality_bins[i+1])
                quality_counts.append(count)

            plt.bar(quality_labels, quality_counts, color='lightblue', edgecolor='black')
            plt.ylabel('Number of Matches')
            plt.title('Match Quality Distribution')
            plt.xticks(rotation=45)

        plt.tight_layout()

        # Save visualization
        viz_path = self.output_dir / "validation_visualizations.png"
        plt.savefig(viz_path, dpi=300, bbox_inches='tight')
        plt.close()

        logging.info(f"Visualizations saved to {viz_path}")

    def generate_detailed_report(self) -> str:
        """Generate comprehensive validation report"""
        if not self.validation_results:
            logging.warning("No validation results found. Run evaluation first.")
            return ""

        # Find best configuration
        best_metrics = max(self.validation_results['threshold_analysis'],
                          key=lambda x: x['f1_score'])

        report = f"""
# KPI Extraction Validation Report
Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

## Dataset Overview
- **Manual KPIs**: {len(self.manual_df)} annotations
- **Auto KPIs**: {len(self.auto_df)} extractions
- **Manual file**: {self.manual_path}
- **Auto file**: {self.auto_path}

## Best Performance Configuration
- **Similarity Type**: {best_metrics['similarity_type']}
- **Threshold**: {best_metrics['threshold']}
- **Precision**: {best_metrics['precision']:.3f}
- **Recall**: {best_metrics['recall']:.3f}
- **F1 Score**: {best_metrics['f1_score']:.3f}

## Detailed Metrics
- **True Positives**: {best_metrics['true_positives']}
- **False Positives**: {best_metrics['false_positives']}
- **False Negatives**: {best_metrics['false_negatives']}
- **Match Rate**: {best_metrics['match_rate']:.3f}

## Error Analysis

### False Positives ({len(self.validation_results['false_positives'])})
KPIs extracted automatically but not in manual annotations:
"""

        # Add false positives
        for i, fp in enumerate(self.validation_results['false_positives'][:10], 1):
            report += f"\n{i}. **{fp['category']}** | {fp['kpi_text']}\n"

        if len(self.validation_results['false_positives']) > 10:
            report += f"\n... and {len(self.validation_results['false_positives']) - 10} more\n"

        report += f"""
### False Negatives ({len(self.validation_results['false_negatives'])})
KPIs in manual annotations but missed by extraction:
"""

        # Add false negatives
        for i, fn in enumerate(self.validation_results['false_negatives'][:10], 1):
            report += f"\n{i}. **{fn['category']}** | {fn['kpi_text']}\n"

        if len(self.validation_results['false_negatives']) > 10:
            report += f"\n... and {len(self.validation_results['false_negatives']) - 10} more\n"

        # Category analysis
        if 'category_analysis' in self.validation_results:
            report += "\n## Category-wise Performance\n\n"
            report += "| Category | Manual | Auto | Matches | Precision | Recall | F1 |\n"
            report += "|----------|--------|------|---------|-----------|--------|----|\\n"

            for category, metrics in self.validation_results['category_analysis'].items():
                report += f"| {category[:20]} | {metrics['manual_count']} | {metrics['auto_count']} | {metrics['matches']} | {metrics['precision']:.3f} | {metrics['recall']:.3f} | {metrics['f1_score']:.3f} |\n"

        # Theme analysis
        if 'theme_analysis' in self.validation_results:
            report += "\n## Theme-wise Coverage\n\n"
            report += "| Theme | Manual Count | Auto Count | Coverage |\n"
            report += "|-------|--------------|------------|----------|\n"

            for theme, metrics in self.validation_results['theme_analysis'].items():
                report += f"| {theme} | {metrics['manual_count']} | {metrics['auto_count']} | {metrics['coverage']:.3f} |\n"

        # Recommendations
        report += f"""
## Recommendations

### Strengths
- Overall F1 Score: {best_metrics['f1_score']:.3f}
- Precision: {best_metrics['precision']:.3f} (low false positive rate)
- Recall: {best_metrics['recall']:.3f} (good coverage)

### Areas for Improvement
"""

        if best_metrics['precision'] < 0.8:
            report += "- **Precision**: Consider stricter filtering to reduce false positives\n"

        if best_metrics['recall'] < 0.8:
            report += "- **Recall**: Improve extraction to catch more manual KPIs\n"

        if best_metrics['f1_score'] < 0.7:
            report += "- **Overall Performance**: Significant room for improvement in both precision and recall\n"

        # Source-specific recommendations
        if 'source_type' in self.auto_df.columns:
            text_kpis = len(self.auto_df[self.auto_df['source_type'] == 'text'])
            image_kpis = len(self.auto_df[self.auto_df['source_type'] == 'image'])

            report += f"""
### Source Type Analysis
- **Text/Table KPIs**: {text_kpis}
- **Image/Chart KPIs**: {image_kpis}
- **Image Coverage**: {image_kpis / (text_kpis + image_kpis) * 100:.1f}%
"""

        return report

    def save_results(self):
        """Save all validation results to files"""
        # Save detailed results as JSON
        results_path = self.output_dir / "validation_results.json"
        with open(results_path, 'w', encoding='utf-8') as f:
            json.dump(self.validation_results, f, indent=2, ensure_ascii=False, default=str)

        # Save matches as Excel
        if self.validation_results['detailed_matches']:
            matches_df = pd.DataFrame(self.validation_results['detailed_matches'])
            matches_path = self.output_dir / "detailed_matches.xlsx"
            matches_df.to_excel(matches_path, index=False)

        # Save false positives and negatives
        fp_df = pd.DataFrame(self.validation_results['false_positives'])
        fn_df = pd.DataFrame(self.validation_results['false_negatives'])

        with pd.ExcelWriter(self.output_dir / "error_analysis.xlsx") as writer:
            fp_df.to_excel(writer, sheet_name='False_Positives', index=False)
            fn_df.to_excel(writer, sheet_name='False_Negatives', index=False)

        # Save report
        report = self.generate_detailed_report()
        report_path = self.output_dir / "validation_report.md"
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(report)

        # Save metrics summary
        threshold_df = pd.DataFrame(self.validation_results['threshold_analysis'])
        threshold_df.to_excel(self.output_dir / "threshold_analysis.xlsx", index=False)

        logging.info(f"All validation results saved to {self.output_dir}")

        return {
            'results_json': results_path,
            'matches_excel': self.output_dir / "detailed_matches.xlsx",
            'error_analysis': self.output_dir / "error_analysis.xlsx",
            'report_markdown': report_path,
            'threshold_analysis': self.output_dir / "threshold_analysis.xlsx",
            'visualizations': self.output_dir / "validation_visualizations.png"
        }

    def run_full_validation(self) -> Dict[str, any]:
        """Run complete validation pipeline"""
        logging.info("Starting full validation pipeline...")

        # Step 1: Run comprehensive evaluation
        self.run_comprehensive_evaluation()

        # Step 2: Generate visualizations
        self.generate_visualizations()

        # Step 3: Save all results
        saved_files = self.save_results()

        # Step 4: Print summary
        best_metrics = max(self.validation_results['threshold_analysis'],
                          key=lambda x: x['f1_score'])

        print("\n" + "="*60)
        print("KPI EXTRACTION VALIDATION SUMMARY")
        print("="*60)
        print(f"📊 Dataset: {len(self.manual_df)} manual vs {len(self.auto_df)} auto KPIs")
        print(f"🎯 Best F1 Score: {best_metrics['f1_score']:.3f}")
        print(f"📈 Precision: {best_metrics['precision']:.3f}")
        print(f"📉 Recall: {best_metrics['recall']:.3f}")
        print(f"✅ True Positives: {best_metrics['true_positives']}")
        print(f"❌ False Positives: {best_metrics['false_positives']}")
        print(f"⚠️  False Negatives: {best_metrics['false_negatives']}")
        print("="*60)
        print(f"📁 Results saved to: {self.output_dir}")
        print("="*60)

        return {
            'validation_results': self.validation_results,
            'saved_files': saved_files,
            'best_metrics': best_metrics
        }


In [26]:
# ============ 兼容性函数 ============
def extract_kpi_from_chunk(chunk: str) -> List[Dict]:
    """Backward compatibility function"""
    return extract_kpi_from_chunk_universal(chunk)

def process_sustainability_report(pdf_path: str) -> pd.DataFrame:
    """Backward compatibility function for text-only processing"""
    return process_text_only()

def process_sustainability_report_with_images(pdf_path: str) -> pd.DataFrame:
    """Backward compatibility function for full processing"""
    return process_sustainability_report_with_enhanced_images(pdf_path)


In [27]:
# ============ 使用示例 ============
def example_usage():
    """Usage examples"""
    print("=== KPI Extraction Tool Usage Examples ===\n")

    print("1. Full extraction (text + images):")
    print("   df_results = process_sustainability_report_with_enhanced_images(PDF_PATH)")
    print("   save_results(df_results, EXPORT_AUTO_XLSX, PDF_PATH)\n")

    print("2. Text-only extraction:")
    print("   df_results = process_text_only()")
    print("   # Results automatically saved\n")

    print("3. Simple run:")
    print("   run_kpi_extraction()  # Complete pipeline with validation\n")

    print("4. Debug single component:")
    print("   test_text_extraction_only()  # Test first 3 chunks")
    print("   debug_single_image_analysis('path/to/image.jpg')\n")

    print("5. Install dependencies:")
    print("   install_dependencies()  # Install all required packages\n")


In [28]:
class MetadataValidationExtension:
    """元数据验证扩展 - 验证KPI文本以外的字段准确性"""

    def __init__(self, validation_pipeline: KPIValidationPipeline):
        """
        基于现有的KPIValidationPipeline扩展元数据验证功能

        Args:
            validation_pipeline: 现有的验证管道实例
        """
        self.main_validator = validation_pipeline
        self.metadata_results = {}

        # 需要验证的元数据字段配置
        self.metadata_fields = {
            # 文档相关字段
            'document_fields': {
                'PDF file name': {'weight': 0.05, 'type': 'exact'},
                'Title of the report': {'weight': 0.05, 'type': 'exact'},
                'Absolute Page Number': {'weight': 0.15, 'type': 'exact'},  # 您提到的问题字段
                'Impacted Stakeholder': {'weight': 0.10, 'type': 'similarity'}
            },

            # KPI分类字段
            'classification_fields': {
                'kpi_theme': {'weight': 0.15, 'type': 'exact'},  # 您提到的问题字段
                'kpi_category': {'weight': 0.15, 'type': 'similarity'}
            },

            # 数值相关字段
            'quantitative_fields': {
                'quantitative_value': {'weight': 0.20, 'type': 'numerical'},  # 您提到的问题字段
                'unit': {'weight': 0.10, 'type': 'similarity'},
                'time_period': {'weight': 0.05, 'type': 'similarity'}
            },

            # 其他分析字段
            'analysis_fields': {
                'target_or_actual': {'weight': 0.05, 'type': 'exact'},
                'source_type': {'weight': 0.05, 'type': 'exact'},
                'chart_type': {'weight': 0.03, 'type': 'similarity'},
                'estimation_confidence': {'weight': 0.02, 'type': 'exact'},
                'chart_title': {'weight': 0.03, 'type': 'similarity'},
                'data_source': {'weight': 0.02, 'type': 'similarity'},
                'image_type': {'weight': 0.02, 'type': 'exact'}
            }
        }

        # 问题字段标记（您特别关注的字段）
        self.problematic_fields = {
            'Absolute Page Number': 'page_number_issues',
            'kpi_theme': 'theme_classification_issues',
            'quantitative_value': 'value_extraction_issues'
        }

        logging.info("元数据验证扩展初始化完成")

    def validate_single_field(self, manual_value, auto_value, field_name: str, validation_type: str) -> Dict[str, any]:
        """
        验证单个字段的准确性

        Args:
            manual_value: 手动标注值
            auto_value: 自动提取值
            field_name: 字段名称
            validation_type: 验证类型 ('exact', 'similarity', 'numerical')

        Returns:
            验证结果字典
        """
        result = {
            'field_name': field_name,
            'manual_value': manual_value,
            'auto_value': auto_value,
            'validation_type': validation_type,
            'is_correct': False,
            'score': 0.0,
            'error_type': None,
            'notes': ""
        }

        # 处理空值情况
        manual_clean = str(manual_value).strip() if pd.notna(manual_value) else ""
        auto_clean = str(auto_value).strip() if pd.notna(auto_value) else ""

        if not manual_clean and not auto_clean:
            result.update({'is_correct': True, 'score': 1.0, 'notes': 'Both values empty'})
            return result
        elif not manual_clean or not auto_clean:
            result.update({'error_type': 'missing_value', 'notes': 'One value is missing'})
            return result

        # 根据验证类型进行比较
        if validation_type == 'exact':
            is_match = manual_clean.lower() == auto_clean.lower()
            result.update({
                'is_correct': is_match,
                'score': 1.0 if is_match else 0.0,
                'error_type': None if is_match else 'exact_mismatch'
            })

        elif validation_type == 'similarity':
            # 使用主验证器的文本相似度算法
            similarity_scores = self.main_validator.calculate_text_similarity(manual_clean, auto_clean)
            similarity = similarity_scores['combined']

            # 对于元数据，使用更高的相似度阈值
            threshold = 0.8
            is_match = similarity >= threshold

            result.update({
                'is_correct': is_match,
                'score': similarity,
                'similarity_details': similarity_scores,
                'error_type': None if is_match else 'similarity_mismatch',
                'notes': f'Similarity: {similarity:.3f}'
            })

        elif validation_type == 'numerical':
            try:
                # 标准化数值格式
                manual_num = self._normalize_numerical_value(manual_clean)
                auto_num = self._normalize_numerical_value(auto_clean)

                if manual_num is not None and auto_num is not None:
                    # 允许小幅度差异（适应提取中的舍入误差）
                    tolerance = 0.01 if abs(manual_num) < 10 else abs(manual_num) * 0.001
                    is_match = abs(manual_num - auto_num) <= tolerance

                    result.update({
                        'is_correct': is_match,
                        'score': 1.0 if is_match else max(0.0, 1.0 - abs(manual_num - auto_num) / max(abs(manual_num), 1)),
                        'error_type': None if is_match else 'numerical_mismatch',
                        'manual_parsed': manual_num,
                        'auto_parsed': auto_num,
                        'notes': f'Manual: {manual_num}, Auto: {auto_num}'
                    })
                else:
                    result.update({
                        'error_type': 'numerical_parse_error',
                        'notes': f'Failed to parse numerical values: "{manual_clean}" vs "{auto_clean}"'
                    })
            except Exception as e:
                result.update({
                    'error_type': 'numerical_validation_error',
                    'notes': f'Numerical validation failed: {str(e)}'
                })

        return result

    def _normalize_numerical_value(self, value_str: str) -> float:
        """标准化数值字符串为float"""
        if not value_str:
            return None

        try:
            # 移除常见的非数字字符，保留数字、小数点、负号
            cleaned = re.sub(r'[^\d\.\-\+]', '', str(value_str))
            if cleaned:
                return float(cleaned)
        except:
            pass

        # 尝试提取第一个数字
        numbers = re.findall(r'-?\d+\.?\d*', str(value_str))
        if numbers:
            try:
                return float(numbers[0])
            except:
                pass

        return None

    def validate_metadata_for_matched_pairs(self) -> Dict[str, any]:
        """
        对已匹配的KPI对进行元数据验证

        Returns:
            元数据验证结果
        """
        logging.info("开始元数据验证...")

        # 获取主验证器的匹配结果
        if not hasattr(self.main_validator, 'validation_results') or not self.main_validator.validation_results:
            logging.error("主验证器尚未运行，请先运行文本验证")
            return {}

        matched_pairs = self.main_validator.validation_results.get('detailed_matches', [])
        if not matched_pairs:
            logging.warning("没有找到匹配的KPI对，无法进行元数据验证")
            return {}

        metadata_results = {
            'total_pairs': len(matched_pairs),
            'field_results': {},
            'overall_scores': {},
            'problematic_field_analysis': {},
            'detailed_results': []
        }

        # 初始化字段结果统计
        all_fields = {}
        for category, fields in self.metadata_fields.items():
            all_fields.update(fields)

        for field_name in all_fields.keys():
            metadata_results['field_results'][field_name] = {
                'total_comparisons': 0,
                'correct_count': 0,
                'total_score': 0.0,
                'error_types': {},
                'examples': {'correct': [], 'incorrect': []}
            }

        # 对每个匹配对进行元数据验证
        for pair_idx, match_pair in enumerate(matched_pairs):
            manual_idx = match_pair['manual_idx']
            auto_idx = match_pair['auto_idx']
            text_similarity = match_pair['similarity']

            # 获取完整的KPI记录
            manual_kpi = self.main_validator.manual_df.iloc[manual_idx]
            auto_kpi = self.main_validator.auto_df.iloc[auto_idx]

            pair_results = {
                'pair_index': pair_idx,
                'manual_idx': manual_idx,
                'auto_idx': auto_idx,
                'text_similarity': text_similarity,
                'manual_text': match_pair['manual_text'],
                'auto_text': match_pair['auto_text'],
                'field_validations': {},
                'pair_metadata_score': 0.0
            }

            total_weight = 0.0
            weighted_score = 0.0

            # 验证每个元数据字段
            for field_name, field_config in all_fields.items():
                manual_value = manual_kpi.get(field_name, '')
                auto_value = auto_kpi.get(field_name, '')

                field_result = self.validate_single_field(
                    manual_value, auto_value, field_name, field_config['type']
                )

                pair_results['field_validations'][field_name] = field_result

                # 更新字段统计
                field_stats = metadata_results['field_results'][field_name]
                field_stats['total_comparisons'] += 1
                field_stats['total_score'] += field_result['score']

                if field_result['is_correct']:
                    field_stats['correct_count'] += 1
                    if len(field_stats['examples']['correct']) < 3:
                        field_stats['examples']['correct'].append({
                            'manual': manual_value,
                            'auto': auto_value,
                            'pair_idx': pair_idx
                        })
                else:
                    error_type = field_result.get('error_type', 'unknown')
                    field_stats['error_types'][error_type] = field_stats['error_types'].get(error_type, 0) + 1

                    if len(field_stats['examples']['incorrect']) < 3:
                        field_stats['examples']['incorrect'].append({
                            'manual': manual_value,
                            'auto': auto_value,
                            'error_type': error_type,
                            'pair_idx': pair_idx,
                            'notes': field_result.get('notes', '')
                        })

                # 计算加权分数
                weight = field_config['weight']
                weighted_score += field_result['score'] * weight
                total_weight += weight

            # 计算该对KPI的元数据总分
            if total_weight > 0:
                pair_results['pair_metadata_score'] = weighted_score / total_weight

            metadata_results['detailed_results'].append(pair_results)

        # 计算整体分数
        self._calculate_overall_metadata_scores(metadata_results, all_fields)

        # 分析问题字段
        self._analyze_problematic_fields(metadata_results)

        self.metadata_results = metadata_results
        logging.info(f"元数据验证完成，共验证 {len(matched_pairs)} 对KPI")

        return metadata_results

    def _calculate_overall_metadata_scores(self, metadata_results: Dict, all_fields: Dict):
        """计算整体元数据分数"""
        overall_scores = metadata_results['overall_scores']

        # 按类别计算分数
        for category, fields in self.metadata_fields.items():
            category_score = 0.0
            category_weight = 0.0

            for field_name, field_config in fields.items():
                field_stats = metadata_results['field_results'][field_name]
                if field_stats['total_comparisons'] > 0:
                    field_accuracy = field_stats['correct_count'] / field_stats['total_comparisons']
                    field_avg_score = field_stats['total_score'] / field_stats['total_comparisons']

                    # 使用平均分数而不是简单的正确率
                    category_score += field_avg_score * field_config['weight']
                    category_weight += field_config['weight']

            if category_weight > 0:
                overall_scores[category] = category_score / category_weight
            else:
                overall_scores[category] = 0.0

        # 计算总体元数据分数
        total_score = 0.0
        total_weight = 0.0

        for field_name, field_config in all_fields.items():
            field_stats = metadata_results['field_results'][field_name]
            if field_stats['total_comparisons'] > 0:
                field_avg_score = field_stats['total_score'] / field_stats['total_comparisons']
                total_score += field_avg_score * field_config['weight']
                total_weight += field_config['weight']

        if total_weight > 0:
            overall_scores['overall_metadata_score'] = total_score / total_weight
        else:
            overall_scores['overall_metadata_score'] = 0.0

    def _analyze_problematic_fields(self, metadata_results: Dict):
        """分析问题字段的详细情况"""
        problematic_analysis = metadata_results['problematic_field_analysis']

        for field_name, issue_type in self.problematic_fields.items():
            if field_name in metadata_results['field_results']:
                field_stats = metadata_results['field_results'][field_name]

                if field_stats['total_comparisons'] > 0:
                    accuracy = field_stats['correct_count'] / field_stats['total_comparisons']
                    avg_score = field_stats['total_score'] / field_stats['total_comparisons']

                    problematic_analysis[field_name] = {
                        'issue_type': issue_type,
                        'accuracy': accuracy,
                        'average_score': avg_score,
                        'total_errors': field_stats['total_comparisons'] - field_stats['correct_count'],
                        'error_breakdown': field_stats['error_types'],
                        'severity': self._assess_field_severity(accuracy, field_name),
                        'improvement_priority': self._get_improvement_priority(field_name, accuracy)
                    }

    def _assess_field_severity(self, accuracy: float, field_name: str) -> str:
        """评估字段问题的严重程度"""
        if accuracy >= 0.9:
            return "low"
        elif accuracy >= 0.7:
            return "medium"
        elif accuracy >= 0.5:
            return "high"
        else:
            return "critical"

    def _get_improvement_priority(self, field_name: str, accuracy: float) -> str:
        """获取改进优先级"""
        field_importance = {
            'Absolute Page Number': 'medium',  # 您说会升级
            'kpi_theme': 'high',  # 重要分类字段
            'quantitative_value': 'critical'  # 最重要的数值字段
        }

        importance = field_importance.get(field_name, 'low')

        if accuracy < 0.5 and importance in ['high', 'critical']:
            return 'urgent'
        elif accuracy < 0.7 and importance == 'critical':
            return 'high'
        elif accuracy < 0.8 and importance in ['high', 'critical']:
            return 'medium'
        else:
            return 'low'

    def generate_metadata_report(self) -> str:
        """生成元数据验证报告"""
        if not self.metadata_results:
            return "No metadata validation results available. Please run validate_metadata_for_matched_pairs() first."

        results = self.metadata_results
        overall_scores = results['overall_scores']

        report = f"""
# 元数据验证报告 (KPI文本验证独立报告)

## 验证概览
- **验证KPI对数**: {results['total_pairs']}
- **总体元数据分数**: {overall_scores.get('overall_metadata_score', 0):.3f}
- **验证时间**: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

## 分类得分

### 文档相关字段 ({overall_scores.get('document_fields', 0):.3f})
"""

        # 添加各类别详细分数
        for category, category_score in overall_scores.items():
            if category.endswith('_fields'):
                category_name = category.replace('_', ' ').title()
                report += f"- **{category_name}**: {category_score:.3f}\n"

        report += "\n## 字段详细分析\n\n"
        report += "| 字段名 | 准确率 | 平均分数 | 错误数 | 主要错误类型 |\n"
        report += "|--------|--------|----------|--------|-------------|\n"

        # 按准确率排序显示字段
        field_items = list(results['field_results'].items())
        field_items.sort(key=lambda x: x[1]['correct_count'] / max(x[1]['total_comparisons'], 1))

        for field_name, field_stats in field_items:
            if field_stats['total_comparisons'] > 0:
                accuracy = field_stats['correct_count'] / field_stats['total_comparisons']
                avg_score = field_stats['total_score'] / field_stats['total_comparisons']
                error_count = field_stats['total_comparisons'] - field_stats['correct_count']

                main_error = max(field_stats['error_types'].items(),
                               key=lambda x: x[1], default=('none', 0))[0]

                report += f"| {field_name} | {accuracy:.3f} | {avg_score:.3f} | {error_count} | {main_error} |\n"

        # 问题字段重点分析
        if results['problematic_field_analysis']:
            report += "\n## 重点关注字段分析\n\n"

            for field_name, analysis in results['problematic_field_analysis'].items():
                report += f"### {field_name} ({analysis['issue_type']})\n"
                report += f"- **准确率**: {analysis['accuracy']:.3f}\n"
                report += f"- **平均分数**: {analysis['average_score']:.3f}\n"
                report += f"- **错误数**: {analysis['total_errors']}\n"
                report += f"- **严重程度**: {analysis['severity']}\n"
                report += f"- **改进优先级**: {analysis['improvement_priority']}\n"

                if analysis['error_breakdown']:
                    report += "- **错误类型分布**:\n"
                    for error_type, count in analysis['error_breakdown'].items():
                        report += f"  - {error_type}: {count}\n"
                report += "\n"

        # 改进建议
        report += "## 改进建议\n\n"

        critical_fields = [name for name, analysis in results['problematic_field_analysis'].items()
                          if analysis['improvement_priority'] in ['urgent', 'high']]

        if critical_fields:
            report += "### 优先改进字段:\n"
            for field in critical_fields:
                analysis = results['problematic_field_analysis'][field]
                report += f"- **{field}**: {analysis['issue_type']} (优先级: {analysis['improvement_priority']})\n"

        report += "\n### 具体建议:\n"
        if 'Absolute Page Number' in results['problematic_field_analysis']:
            report += "- **页码提取**: 考虑改进PDF页码识别算法\n"
        if 'kpi_theme' in results['problematic_field_analysis']:
            report += "- **主题分类**: 优化KPI主题分类逻辑\n"
        if 'quantitative_value' in results['problematic_field_analysis']:
            report += "- **数值提取**: 加强数值识别和标准化处理\n"

        return report

    def save_metadata_validation_results(self, output_dir: str = None):
        """保存元数据验证结果"""
        if not self.metadata_results:
            logging.warning("No metadata results to save")
            return {}

        if output_dir is None:
            output_dir = self.main_validator.output_dir / "metadata_validation"

        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        saved_files = {}

        # 1. 保存详细结果JSON
        results_file = output_path / "metadata_validation_results.json"
        with open(results_file, 'w', encoding='utf-8') as f:
            json.dump(self.metadata_results, f, indent=2, ensure_ascii=False, default=str)
        saved_files['detailed_json'] = results_file

        # 2. 保存字段分析Excel
        field_analysis_data = []
        for field_name, field_stats in self.metadata_results['field_results'].items():
            if field_stats['total_comparisons'] > 0:
                accuracy = field_stats['correct_count'] / field_stats['total_comparisons']
                avg_score = field_stats['total_score'] / field_stats['total_comparisons']

                field_analysis_data.append({
                    'Field Name': field_name,
                    'Total Comparisons': field_stats['total_comparisons'],
                    'Correct Count': field_stats['correct_count'],
                    'Accuracy': accuracy,
                    'Average Score': avg_score,
                    'Error Count': field_stats['total_comparisons'] - field_stats['correct_count'],
                    'Main Error Type': max(field_stats['error_types'].items(),
                                         key=lambda x: x[1], default=('none', 0))[0]
                })

        field_df = pd.DataFrame(field_analysis_data)
        field_df = field_df.sort_values('Accuracy')

        excel_file = output_path / "metadata_field_analysis.xlsx"
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            field_df.to_excel(writer, sheet_name='Field Analysis', index=False)

            # 问题字段详细分析
            if self.metadata_results['problematic_field_analysis']:
                prob_data = []
                for field_name, analysis in self.metadata_results['problematic_field_analysis'].items():
                    prob_data.append({
                        'Field Name': field_name,
                        'Issue Type': analysis['issue_type'],
                        'Accuracy': analysis['accuracy'],
                        'Average Score': analysis['average_score'],
                        'Total Errors': analysis['total_errors'],
                        'Severity': analysis['severity'],
                        'Improvement Priority': analysis['improvement_priority']
                    })

                prob_df = pd.DataFrame(prob_data)
                prob_df.to_excel(writer, sheet_name='Problematic Fields', index=False)

        saved_files['field_analysis_excel'] = excel_file

        # 3. 保存元数据报告
        report = self.generate_metadata_report()
        report_file = output_path / "metadata_validation_report.md"
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write(report)
        saved_files['report_markdown'] = report_file

        logging.info(f"元数据验证结果已保存到: {output_path}")
        return saved_files

# 集成函数：在主验证流程中添加元数据验证
def run_complete_validation_with_metadata(df_auto: pd.DataFrame, manual_xlsx_path: str,
                                         output_dir: str = "complete_validation") -> Dict[str, any]:
    """
    运行完整验证（文本验证 + 元数据验证）

    Args:
        df_auto: 自动提取的KPI DataFrame
        manual_xlsx_path: 手动标注文件路径
        output_dir: 输出目录

    Returns:
        完整验证结果
    """
    logging.info("开始运行完整验证（文本 + 元数据）...")

    # Step 1: 运行文本验证
    print("🔍 Step 1: 运行文本验证...")
    text_validation_results = enhanced_compare_with_manual_kpis(
        df_auto, manual_xlsx_path, f"{output_dir}/text_validation"
    )

    if not text_validation_results:
        logging.error("文本验证失败，无法继续元数据验证")
        return {}

    # Step 2: 运行元数据验证
    print("📊 Step 2: 运行元数据验证...")

    # 获取文本验证器实例
    temp_auto_path = Path(f"{output_dir}/text_validation") / "temp_auto_kpis.xlsx"
    if not temp_auto_path.exists():
        temp_auto_path = Path(output_dir) / "temp_auto_kpis.xlsx"
        df_auto.to_excel(temp_auto_path, index=False)

    main_validator = KPIValidationPipeline(
        manual_excel_path=manual_xlsx_path,
        auto_excel_path=str(temp_auto_path),
        output_dir=f"{output_dir}/text_validation"
    )

    # 确保文本验证已运行
    if not hasattr(main_validator, 'validation_results') or not main_validator.validation_results:
        main_validator.run_comprehensive_evaluation()

    # 创建元数据验证扩展
    metadata_validator = MetadataValidationExtension(main_validator)
    metadata_results = metadata_validator.validate_metadata_for_matched_pairs()

    # 保存元数据验证结果
    metadata_saved_files = metadata_validator.save_metadata_validation_results(f"{output_dir}/metadata_validation")

    # Step 3: 生成综合报告
    print("📋 Step 3: 生成综合报告...")
    complete_results = {
        'text_validation': text_validation_results,
        'metadata_validation': metadata_results,
        'saved_files': {
            'text_validation_files': text_validation_results.get('saved_files', {}),
            'metadata_validation_files': metadata_saved_files
        }
    }

    # 打印摘要
    text_best = text_validation_results.get('best_metrics', {})
    metadata_overall = metadata_results.get('overall_scores', {})

    print(f"\n{'='*60}")
    print("完整验证结果摘要")
    print(f"{'='*60}")
    print(f"📝 文本验证 F1 分数: {text_best.get('f1_score', 0):.3f}")
    print(f"📊 元数据总体分数: {metadata_overall.get('overall_metadata_score', 0):.3f}")
    print(f"🎯 验证KPI对数: {metadata_results.get('total_pairs', 0)}")

    print(f"\n📂 元数据分类得分:")
    for category, score in metadata_overall.items():
        if category.endswith('_fields'):
            category_name = category.replace('_', ' ').title()
            print(f"   {category_name}: {score:.3f}")

    # 显示问题字段
    if metadata_results.get('problematic_field_analysis'):
        print(f"\n⚠️  需要关注的字段:")
        for field_name, analysis in metadata_results['problematic_field_analysis'].items():
            print(f"   {field_name}: {analysis['accuracy']:.3f} (优先级: {analysis['improvement_priority']})")

    print(f"\n📁 完整结果保存在: {output_dir}")
    print(f"{'='*60}")

    # 清理临时文件
    if temp_auto_path.exists():
        temp_auto_path.unlink()

    return complete_results

# 便捷使用函数
def add_metadata_validation_to_existing_pipeline():
    """为现有验证流程添加元数据验证的示例函数"""

    def enhanced_main_with_metadata():
        """增强版main函数，包含元数据验证"""
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - %(levelname)s: %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S"
        )

        try:
            if not os.path.exists(PDF_PATH):
                logging.error(f"PDF file not found: {PDF_PATH}")
                return

            # 原有的KPI提取流程
            df_auto = process_sustainability_report_with_enhanced_images(PDF_PATH)
            save_results(df_auto, EXPORT_AUTO_XLSX, PDF_PATH)
            logging.info(f"KPI extraction completed: {len(df_auto)} KPIs extracted")

            # 增强的验证流程（文本 + 元数据）
            if MANUAL_XLSX and Path(MANUAL_XLSX).exists():
                print("\n🔍 Running complete validation (text + metadata)...")
                complete_validation_results = run_complete_validation_with_metadata(
                    df_auto, MANUAL_XLSX, "complete_validation"
                )

                if complete_validation_results:
                    print("✅ Complete validation finished with detailed analysis!")
                    print(f"📁 Detailed results saved to: complete_validation/")
                else:
                    print("⚠️ Validation encountered issues")
            else:
                logging.info("Manual KPI file not found, skipping validation.")

            # 显示提取摘要（保持原有逻辑）
            if not df_auto.empty:
                print(f"\n=== Extraction Summary ===")
                print(f"Total KPIs extracted: {len(df_auto)}")

                if 'source_type' in df_auto.columns:
                    source_counts = df_auto['source_type'].value_counts()
                    print(f"From text/tables: {source_counts.get('text', 0)}")
                    print(f"From images/charts: {source_counts.get('image', 0)}")

                if 'kpi_theme' in df_auto.columns:
                    theme_counts = df_auto['kpi_theme'].value_counts()
                    print(f"\nKPI Distribution by Theme:")
                    for theme, count in theme_counts.items():
                        print(f"  {theme}: {count}")
            else:
                print("\nNo KPIs were extracted from the document.")

        except Exception as e:
            logging.error(f"Error in enhanced main execution: {e}")
            import traceback
            traceback.print_exc()

    return enhanced_main_with_metadata


In [32]:
class BatchKPIProcessor:
    """批量KPI处理器 - 支持多个PDF和Manual文件"""

    def __init__(self, base_output_dir: str = "batch_kpi_results"):
        """
        初始化批量处理器

        Args:
            base_output_dir: 批量处理结果的基础目录
        """
        self.base_output_dir = Path(base_output_dir)
        self.base_output_dir.mkdir(exist_ok=True)

        # 存储所有文件配对
        self.file_pairs = []
        self.batch_results = []

        # 创建时间戳用于本次批量处理
        self.batch_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_batch_dir = self.base_output_dir / f"batch_{self.batch_timestamp}"
        self.current_batch_dir.mkdir(exist_ok=True)

        logging.info(f"批量处理器初始化完成，结果保存到: {self.current_batch_dir}")

    def add_file_pair(self, pdf_path: str, manual_path: str, document_name: str = None):
        """
        添加一对PDF和Manual文件

        Args:
            pdf_path: PDF文件路径
            manual_path: Manual标注文件路径
            document_name: 文档名称（可选，默认使用PDF文件名）
        """
        pdf_path = Path(pdf_path)
        manual_path = Path(manual_path)

        # 验证文件存在
        if not pdf_path.exists():
            logging.error(f"PDF文件不存在: {pdf_path}")
            return False

        if not manual_path.exists():
            logging.error(f"Manual文件不存在: {manual_path}")
            return False

        # 自动生成文档名称
        if document_name is None:
            document_name = pdf_path.stem

        file_pair = {
            'pdf_path': str(pdf_path),
            'manual_path': str(manual_path),
            'document_name': document_name,
            'doc_id': len(self.file_pairs) + 1
        }

        self.file_pairs.append(file_pair)
        logging.info(f"添加文件对 {len(self.file_pairs)}: {document_name}")
        return True

    def add_multiple_pairs_from_directory(self, pdf_dir: str, manual_dir: str,
                                         pdf_pattern: str = "*.pdf",
                                         manual_pattern: str = "*.xlsx"):
        """
        从目录批量添加文件对（按文件名匹配）

        Args:
            pdf_dir: PDF文件目录
            manual_dir: Manual文件目录
            pdf_pattern: PDF文件匹配模式
            manual_pattern: Manual文件匹配模式
        """
        pdf_dir = Path(pdf_dir)
        manual_dir = Path(manual_dir)

        if not pdf_dir.exists() or not manual_dir.exists():
            logging.error(f"目录不存在: {pdf_dir} 或 {manual_dir}")
            return 0

        # 获取所有PDF文件
        pdf_files = list(pdf_dir.glob(pdf_pattern))
        added_count = 0

        for pdf_file in pdf_files:
            # 尝试找到对应的Manual文件
            base_name = pdf_file.stem

            # 尝试多种匹配模式
            possible_manual_names = [
                f"{base_name}.xlsx",
                f"{base_name}_manual.xlsx",
                f"manual_{base_name}.xlsx",
                f"{base_name}.xls"
            ]

            manual_file = None
            for manual_name in possible_manual_names:
                potential_manual = manual_dir / manual_name
                if potential_manual.exists():
                    manual_file = potential_manual
                    break

            if manual_file:
                if self.add_file_pair(str(pdf_file), str(manual_file), base_name):
                    added_count += 1
            else:
                logging.warning(f"未找到 {base_name} 对应的Manual文件")

        logging.info(f"从目录批量添加了 {added_count} 个文件对")
        return added_count

    def list_file_pairs(self):
        """显示所有已添加的文件对"""
        if not self.file_pairs:
            print("❌ 没有添加任何文件对")
            return

        print(f"\n📋 已添加的文件对 (共 {len(self.file_pairs)} 对):")
        print("-" * 80)
        for pair in self.file_pairs:
            print(f"{pair['doc_id']:2d}. 文档: {pair['document_name']}")
            print(f"    PDF:    {pair['pdf_path']}")
            print(f"    Manual: {pair['manual_path']}")
            print()

    def process_single_document(self, file_pair: Dict) -> Dict:
        """
        处理单个文档（PDF + Manual）

        Args:
            file_pair: 文件对信息

        Returns:
            处理结果字典
        """
        doc_name = file_pair['document_name']
        pdf_path = file_pair['pdf_path']
        manual_path = file_pair['manual_path']
        doc_id = file_pair['doc_id']

        print(f"\n{'='*60}")
        print(f"📄 处理文档 {doc_id}/{len(self.file_pairs)}: {doc_name}")
        print(f"{'='*60}")

        # 为每个文档创建独立目录
        doc_output_dir = self.current_batch_dir / f"doc_{doc_id}_{doc_name}"
        doc_output_dir.mkdir(exist_ok=True)

        start_time = time.time()
        result = {
            'doc_id': doc_id,
            'document_name': doc_name,
            'pdf_path': pdf_path,
            'manual_path': manual_path,
            'output_dir': str(doc_output_dir),
            'start_time': datetime.now().isoformat(),
            'status': 'processing'
        }

        try:
            # Step 1: 临时修改全局PDF路径
            global PDF_PATH
            original_pdf_path = PDF_PATH
            PDF_PATH = pdf_path

            print(f"📊 Step 1: 提取KPI from {Path(pdf_path).name}...")

            # Step 2: 运行KPI提取
            df_auto = process_sustainability_report_with_enhanced_images(PDF_PATH)

            # Step 3: 保存自动提取结果
            auto_excel_path = doc_output_dir / f"{doc_name}_auto_kpis.xlsx"
            save_results(df_auto, str(auto_excel_path), PDF_PATH)

            print(f"✅ 提取完成: {len(df_auto)} KPIs")

            # Step 4: 运行验证
            print(f"🔍 Step 2: 运行验证 against {Path(manual_path).name}...")
            validation_output_dir = doc_output_dir / "validation"
            validation_results = enhanced_compare_with_manual_kpis(
                df_auto, manual_path, str(validation_output_dir)
            )

            # Step 5: 收集结果
            processing_time = time.time() - start_time

            result.update({
                'status': 'completed',
                'processing_time_seconds': processing_time,
                'extracted_kpis_count': len(df_auto),
                'auto_excel_path': str(auto_excel_path),
                'validation_output_dir': str(validation_output_dir),
                'end_time': datetime.now().isoformat()
            })

            # 添加验证指标
            if validation_results and 'best_metrics' in validation_results:
                metrics = validation_results['best_metrics']
                result.update({
                    'validation_f1_score': metrics.get('f1_score', 0),
                    'validation_precision': metrics.get('precision', 0),
                    'validation_recall': metrics.get('recall', 0),
                    'true_positives': metrics.get('true_positives', 0),
                    'false_positives': metrics.get('false_positives', 0),
                    'false_negatives': metrics.get('false_negatives', 0)
                })

                print(f"🎯 验证完成:")
                print(f"   F1 Score: {metrics.get('f1_score', 0):.3f}")
                print(f"   Precision: {metrics.get('precision', 0):.3f}")
                print(f"   Recall: {metrics.get('recall', 0):.3f}")

            print(f"⏱️  处理耗时: {processing_time:.1f}秒")
            print(f"📁 结果保存到: {doc_output_dir}")

            # 恢复原始PDF路径
            PDF_PATH = original_pdf_path

        except Exception as e:
            processing_time = time.time() - start_time
            error_msg = str(e)

            result.update({
                'status': 'failed',
                'processing_time_seconds': processing_time,
                'error_message': error_msg,
                'end_time': datetime.now().isoformat()
            })

            print(f"❌ 处理失败: {error_msg}")

            # 恢复原始PDF路径
            PDF_PATH = original_pdf_path

            # 保存错误日志
            error_log_path = doc_output_dir / "error_log.txt"
            with open(error_log_path, 'w', encoding='utf-8') as f:
                f.write(f"文档: {doc_name}\n")
                f.write(f"错误时间: {datetime.now()}\n")
                f.write(f"错误信息: {error_msg}\n")
                f.write(f"PDF路径: {pdf_path}\n")
                f.write(f"Manual路径: {manual_path}\n")

        return result

    def run_batch_processing(self, max_workers: int = 1):
        """
        运行批量处理

        Args:
            max_workers: 最大并发处理数（建议保持为1，避免API限制）
        """
        if not self.file_pairs:
            print("❌ 没有要处理的文件对")
            return

        print(f"\n🚀 开始批量处理 {len(self.file_pairs)} 个文档...")
        print(f"📁 结果将保存到: {self.current_batch_dir}")

        batch_start_time = time.time()

        # 处理每个文档
        for file_pair in self.file_pairs:
            result = self.process_single_document(file_pair)
            self.batch_results.append(result)

            # 实时保存进度（防止中断丢失结果）
            self.save_batch_progress()

        # 生成最终报告
        batch_total_time = time.time() - batch_start_time
        self.generate_batch_summary(batch_total_time)

        print(f"\n🎉 批量处理完成!")
        print(f"⏱️  总耗时: {batch_total_time:.1f}秒 ({batch_total_time/60:.1f}分钟)")
        print(f"📊 处理统计: {self.get_batch_statistics()}")
        print(f"📁 完整结果查看: {self.current_batch_dir}")

    def save_batch_progress(self):
        """保存批量处理进度"""
        progress_file = self.current_batch_dir / "batch_progress.json"
        with open(progress_file, 'w', encoding='utf-8') as f:
            json.dump({
                'batch_timestamp': self.batch_timestamp,
                'file_pairs': self.file_pairs,
                'results': self.batch_results,
                'last_updated': datetime.now().isoformat()
            }, f, indent=2, ensure_ascii=False)

    def get_batch_statistics(self) -> str:
        """获取批量处理统计信息"""
        if not self.batch_results:
            return "无结果"

        total = len(self.batch_results)
        completed = len([r for r in self.batch_results if r['status'] == 'completed'])
        failed = len([r for r in self.batch_results if r['status'] == 'failed'])

        # 计算平均验证指标
        completed_results = [r for r in self.batch_results if r['status'] == 'completed']
        if completed_results:
            avg_f1 = sum(r.get('validation_f1_score', 0) for r in completed_results) / len(completed_results)
            avg_precision = sum(r.get('validation_precision', 0) for r in completed_results) / len(completed_results)
            avg_recall = sum(r.get('validation_recall', 0) for r in completed_results) / len(completed_results)
            total_kpis = sum(r.get('extracted_kpis_count', 0) for r in completed_results)
        else:
            avg_f1 = avg_precision = avg_recall = total_kpis = 0

        return f"""
        成功: {completed}/{total} ({completed/total*100:.1f}%)
        失败: {failed}/{total} ({failed/total*100:.1f}%)
        总KPI数: {total_kpis}
        平均F1: {avg_f1:.3f}
        平均精确率: {avg_precision:.3f}
        平均召回率: {avg_recall:.3f}
        """

    def generate_batch_summary(self, total_time: float):
        """生成批量处理汇总报告"""
        # 1. 保存详细结果到Excel
        results_df = pd.DataFrame(self.batch_results)
        excel_path = self.current_batch_dir / "batch_summary.xlsx"

        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            # 主要结果
            results_df.to_excel(writer, sheet_name='处理结果', index=False)

            # 验证指标汇总
            if not results_df.empty:
                completed_df = results_df[results_df['status'] == 'completed']
                if not completed_df.empty:
                    validation_columns = ['document_name', 'extracted_kpis_count',
                                        'validation_f1_score', 'validation_precision',
                                        'validation_recall', 'true_positives',
                                        'false_positives', 'false_negatives']

                    validation_df = completed_df[validation_columns].copy()
                    validation_df.to_excel(writer, sheet_name='验证指标', index=False)

        # 2. 生成Markdown报告
        report_path = self.current_batch_dir / "batch_report.md"
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(f"""# 批量KPI提取与验证报告

## 处理概览
- **处理时间**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- **批次ID**: {self.batch_timestamp}
- **总文档数**: {len(self.file_pairs)}
- **总耗时**: {total_time:.1f}秒 ({total_time/60:.1f}分钟)

## 处理统计
{self.get_batch_statistics()}

## 详细结果

| 文档ID | 文档名称 | 状态 | KPI数量 | F1分数 | 精确率 | 召回率 | 处理时间(秒) |
|--------|----------|------|---------|--------|--------|--------|-------------|
""")

            for result in self.batch_results:
                f.write(f"| {result['doc_id']} | {result['document_name']} | {result['status']} | "
                       f"{result.get('extracted_kpis_count', 'N/A')} | "
                       f"{result.get('validation_f1_score', 0):.3f} | "
                       f"{result.get('validation_precision', 0):.3f} | "
                       f"{result.get('validation_recall', 0):.3f} | "
                       f"{result.get('processing_time_seconds', 0):.1f} |\n")

            if any(r['status'] == 'failed' for r in self.batch_results):
                f.write(f"\n## 失败的文档\n")
                for result in self.batch_results:
                    if result['status'] == 'failed':
                        f.write(f"- **{result['document_name']}**: {result.get('error_message', '未知错误')}\n")

        print(f"📋 批量处理报告生成: {report_path}")
        print(f"📊 详细结果Excel: {excel_path}")

In [33]:
# 修改批量处理器，显示详细的单文档验证信息
class BatchKPIProcessorWithDetailedOutput(BatchKPIProcessor):
    """
    扩展批量处理器，支持详细的验证输出
    """

    def process_single_document(self, file_pair: Dict) -> Dict:
        """
        处理单个文档，显示详细的验证信息
        """
        doc_name = file_pair['document_name']
        pdf_path = file_pair['pdf_path']
        manual_path = file_pair['manual_path']
        doc_id = file_pair['doc_id']

        print(f"\n{'='*80}")
        print(f"📄 Processing Document {doc_id}/{len(self.file_pairs)}: {doc_name}")
        print(f"{'='*80}")

        # 为每个文档创建独立目录
        doc_output_dir = self.current_batch_dir / f"doc_{doc_id}_{doc_name}"
        doc_output_dir.mkdir(exist_ok=True)

        start_time = time.time()
        result = {
            'doc_id': doc_id,
            'document_name': doc_name,
            'pdf_path': pdf_path,
            'manual_path': manual_path,
            'output_dir': str(doc_output_dir),
            'start_time': datetime.now().isoformat(),
            'status': 'processing'
        }

        try:
            # Step 1: 临时修改全局PDF路径
            global PDF_PATH
            original_pdf_path = PDF_PATH
            PDF_PATH = pdf_path

            print(f"📊 Step 1: Extracting KPIs from {Path(pdf_path).name}...")

            # Step 2: 运行KPI提取
            df_auto = process_sustainability_report_with_enhanced_images(PDF_PATH)

            # Step 3: 保存自动提取结果
            auto_excel_path = doc_output_dir / f"{doc_name}_auto_kpis.xlsx"
            save_results(df_auto, str(auto_excel_path), PDF_PATH)

            print(f"✅ Extraction completed: {len(df_auto)} KPIs")

            # Step 4: 运行详细验证
            print(f"\n🔍 Step 2: Running comprehensive validation against {Path(manual_path).name}...")
            validation_output_dir = doc_output_dir / "validation"
            validation_start_time = time.time()

            complete_validation_results = enhanced_compare_with_manual_kpis(
                df_auto, manual_path, str(validation_output_dir)
            )

            validation_time = time.time() - validation_start_time

            # Step 5: 收集结果
            processing_time = time.time() - start_time

            result.update({
                'status': 'completed',
                'processing_time_seconds': processing_time,
                'validation_time_seconds': validation_time,
                'extracted_kpis_count': len(df_auto),
                'auto_excel_path': str(auto_excel_path),
                'validation_output_dir': str(validation_output_dir),
                'end_time': datetime.now().isoformat()
            })

            # 添加详细的验证指标
            if complete_validation_results:
                # 文本验证指标
                if 'text_validation' in complete_validation_results:
                    text_metrics = complete_validation_results['text_validation'].get('best_metrics', {})
                    result.update({
                        'text_validation_f1_score': text_metrics.get('f1_score', 0),
                        'text_validation_precision': text_metrics.get('precision', 0),
                        'text_validation_recall': text_metrics.get('recall', 0),
                        'true_positives': text_metrics.get('true_positives', 0),
                        'false_positives': text_metrics.get('false_positives', 0),
                        'false_negatives': text_metrics.get('false_negatives', 0),
                        'manual_kpis_count': text_metrics.get('total_manual', 0),
                        'auto_kpis_count': text_metrics.get('total_auto', 0)
                    })

                # 元数据验证指标
                if 'metadata_validation' in complete_validation_results:
                    metadata_scores = complete_validation_results['metadata_validation'].get('overall_scores', {})
                    result.update({
                        'metadata_overall_score': metadata_scores.get('overall_metadata_score', 0),
                        'document_fields_score': metadata_scores.get('document_fields', 0),
                        'classification_fields_score': metadata_scores.get('classification_fields', 0),
                        'quantitative_fields_score': metadata_scores.get('quantitative_fields', 0),
                        'analysis_fields_score': metadata_scores.get('analysis_fields', 0)
                    })

                    # 添加问题字段分析
                    problematic_fields = complete_validation_results['metadata_validation'].get('problematic_field_analysis', {})
                    for field_name in ['Absolute Page Number', 'kpi_theme', 'quantitative_value']:
                        if field_name in problematic_fields:
                            field_key = field_name.lower().replace(' ', '_')
                            result[f'{field_key}_accuracy'] = problematic_fields[field_name]['accuracy']
                            result[f'{field_key}_priority'] = problematic_fields[field_name]['improvement_priority']

                # 显示最终摘要
                print(f"\n🎯 Document {doc_id} Verification Completed:")
                print(f"   📊 Dataset: {result.get('manual_kpis_count', 0)} manual vs {result.get('auto_kpis_count', 0)} auto KPIs")
                print(f"   🎯 Text F1 Score: {result.get('text_validation_f1_score', 0):.3f}")
                print(f"   📈 Text Precision: {result.get('text_validation_precision', 0):.3f}")
                print(f"   📉 Text Recall: {result.get('text_validation_recall', 0):.3f}")
                print(f"   ✅ True Positives: {result.get('true_positives', 0)}")
                print(f"   ❌ False Positives: {result.get('false_positives', 0)}")
                print(f"   ⚠️  False Negatives: {result.get('false_negatives', 0)}")
                print(f"   📊 Metadata Overall Score: {result.get('metadata_overall_score', 0):.3f}")

            print(f"⏱️  Processing time: {processing_time:.1f}秒 (validation: {validation_time:.1f}秒)")
            print(f"📁 Results saved to: {doc_output_dir}")

            # 恢复原始PDF路径
            PDF_PATH = original_pdf_path

        except Exception as e:
            processing_time = time.time() - start_time
            error_msg = str(e)

            result.update({
                'status': 'failed',
                'processing_time_seconds': processing_time,
                'error_message': error_msg,
                'end_time': datetime.now().isoformat()
            })

            print(f"❌ Processing failed: {error_msg}")

            # 恢复原始PDF路径
            PDF_PATH = original_pdf_path

            # 保存错误日志
            error_log_path = doc_output_dir / "error_log.txt"
            with open(error_log_path, 'w', encoding='utf-8') as f:
                f.write(f"Document: {doc_name}\n")
                f.write(f"Error time: {datetime.now()}\n")
                f.write(f"Error message: {error_msg}\n")
                f.write(f"PDF Path: {pdf_path}\n")
                f.write(f"Manual Path: {manual_path}\n")

        return result

    def generate_batch_summary(self, total_time: float):
        """生成详细的批量处理汇总报告"""
        # 调用父类方法生成基础报告
        super().generate_batch_summary(total_time)

        # 生成详细的批量摘要
        completed_results = [r for r in self.batch_results if r['status'] == 'completed']

        if completed_results:
            print(f"\n{'='*80}")
            print("BATCH PROCESSING DETAILED SUMMARY")
            print(f"{'='*80}")

            # 计算批量统计
            total_manual_kpis = sum(r.get('manual_kpis_count', 0) for r in completed_results)
            total_auto_kpis = sum(r.get('auto_kpis_count', 0) for r in completed_results)
            total_true_positives = sum(r.get('true_positives', 0) for r in completed_results)
            total_false_positives = sum(r.get('false_positives', 0) for r in completed_results)
            total_false_negatives = sum(r.get('false_negatives', 0) for r in completed_results)

            # 计算平均指标
            avg_f1 = sum(r.get('text_validation_f1_score', 0) for r in completed_results) / len(completed_results)
            avg_precision = sum(r.get('text_validation_precision', 0) for r in completed_results) / len(completed_results)
            avg_recall = sum(r.get('text_validation_recall', 0) for r in completed_results) / len(completed_results)
            avg_metadata_score = sum(r.get('metadata_overall_score', 0) for r in completed_results) / len(completed_results)

            print(f"📊 Batch Dataset Summary:")
            print(f"   Total Documents Processed: {len(completed_results)}")
            print(f"   Total Manual KPIs: {total_manual_kpis}")
            print(f"   Total Auto KPIs: {total_auto_kpis}")
            print(f"   Total True Positives: {total_true_positives}")
            print(f"   Total False Positives: {total_false_positives}")
            print(f"   Total False Negatives: {total_false_negatives}")

            print(f"\n🎯 Batch Average Performance:")
            print(f"   Average Text F1 Score: {avg_f1:.3f}")
            print(f"   Average Text Precision: {avg_precision:.3f}")
            print(f"   Average Text Recall: {avg_recall:.3f}")
            print(f"   Average Metadata Overall Score: {avg_metadata_score:.3f}")

            # 显示各文档表现
            print(f"\n📋 Individual Document Performance:")
            print("="*80)

            for result in completed_results:
                doc_name = result['document_name'][:30]  # 限制长度
                f1 = result.get('text_validation_f1_score', 0)
                precision = result.get('text_validation_precision', 0)
                recall = result.get('text_validation_recall', 0)
                metadata = result.get('metadata_overall_score', 0)
                tp = result.get('true_positives', 0)
                fp = result.get('false_positives', 0)
                fn = result.get('false_negatives', 0)

                print(f"{doc_name:<30} | F1: {f1:.3f} | P: {precision:.3f} | R: {recall:.3f} | Meta: {metadata:.3f} | TP:{tp:2d} FP:{fp:2d} FN:{fn:2d}")

            print("="*80)
            print(f"📁 Detailed results saved to: {self.current_batch_dir}")
            print("="*80)

# 创建详细输出的批量处理器
def create_batch_processor_with_detailed_output():
    """创建支持详细输出的批量处理器"""
    return BatchKPIProcessorWithDetailedOutput()

# 修改集成执行函数
def integrated_main_execution_detailed():
    """集成的主执行函数 - 详细输出版本"""
    print("🚀 Enhanced KPI Extraction System - Detailed Output")
    print("=" * 60)
    print("1. Single PDF processing (detailed text + metadata validation)")
    print("2. Batch PDF processing (detailed text + metadata validation)")
    print("3. Quick batch from directories (detailed output)")
    print("4. Quick metadata validation test")
    print("5. View usage examples")

    try:
        choice = input("Please select processing mode (1-5): ")

        if choice == '1':
            # 单个PDF处理，详细输出
            enhanced_main_with_detailed_output()

        elif choice == '2':
            # 批量处理，详细输出
            processor = create_batch_processor_with_detailed_output()

            # 让用户手动添加文件对
            while True:
                pdf_path = input("Enter PDF file path (press Enter to finish): ").strip()
                if not pdf_path:
                    break
                manual_path = input("Enter corresponding Manual file path: ").strip()
                doc_name = input("Document name (press Enter for default): ").strip() or None

                processor.add_file_pair(pdf_path, manual_path, doc_name)

            if processor.file_pairs:
                processor.list_file_pairs()
                confirm = input(f"\nStart detailed processing of these {len(processor.file_pairs)} documents? (y/n): ")
                if confirm.lower() == 'y':
                    processor.run_batch_processing()
                else:
                    print("Batch processing cancelled.")
            else:
                print("❌ No file pairs added.")

        elif choice == '3':
            # 快速目录批量处理，详细输出
            pdf_dir = input("PDF files directory path: ").strip()
            manual_dir = input("Manual files directory path: ").strip()

            processor = create_batch_processor_with_detailed_output()
            added_count = processor.add_multiple_pairs_from_directory(pdf_dir, manual_dir)

            if added_count == 0:
                print("❌ No matching PDF and Manual file pairs found.")
                return None

            processor.list_file_pairs()
            response = input(f"\nStart detailed processing of these {added_count} documents? (y/n): ")
            if response.lower() == 'y':
                processor.run_batch_processing()
            else:
                print("Batch processing cancelled.")

        elif choice == '4':
            # 快速元数据验证测试
            quick_test_metadata_validation()

        elif choice == '5':
            # 显示使用示例
            metadata_validation_usage_examples()

        else:
            print("Invalid selection, running default single PDF processing")
            enhanced_main_with_detailed_output()

    except KeyboardInterrupt:
        print("\nUser cancelled operation")
    except Exception as e:
        print(f"Execution error: {e}")
        print("Running default single PDF processing")
        enhanced_main_with_detailed_output()

In [34]:
class FileUploadManager:
    """文件上传和管理器"""

    def __init__(self):
        self.uploaded_files = []
        self.pdf_files = []
        self.manual_files = []
        self.file_pairs = []

        # 创建工作目录
        self.work_dir = Path("/content/kpi_files")
        self.work_dir.mkdir(exist_ok=True)

        print(f"📁 工作目录: {self.work_dir}")

    def upload_files_directly(self):
        """直接上传文件到Colab"""
        print("📤 请选择要上传的文件...")
        print("可以同时选择多个PDF和Excel文件")

        uploaded = files.upload()

        for filename, content in uploaded.items():
            file_path = self.work_dir / filename
            with open(file_path, 'wb') as f:
                f.write(content)

            self.uploaded_files.append(str(file_path))
            print(f"✅ 已上传: {filename}")

        self._categorize_files()
        return len(uploaded)

    def mount_google_drive(self):
        """挂载Google Drive"""
        try:
            drive.mount('/content/drive')
            print("✅ Google Drive已挂载")
            print("📁 你的文件在: /content/drive/MyDrive/")
            return True
        except Exception as e:
            print(f"❌ Drive挂载失败: {e}")
            return False

    def scan_drive_directory(self, drive_path: str):
        """扫描Drive目录中的文件"""
        drive_path = Path(drive_path)

        if not drive_path.exists():
            print(f"❌ 目录不存在: {drive_path}")
            return 0

        # 扫描PDF文件
        pdf_files = list(drive_path.glob("*.pdf"))
        excel_files = list(drive_path.glob("*.xlsx")) + list(drive_path.glob("*.xls"))

        print(f"📊 发现文件:")
        print(f"   PDF文件: {len(pdf_files)}个")
        print(f"   Excel文件: {len(excel_files)}个")

        # 复制到工作目录
        for pdf_file in pdf_files:
            dest = self.work_dir / pdf_file.name
            shutil.copy2(pdf_file, dest)
            self.uploaded_files.append(str(dest))
            print(f"📄 复制PDF: {pdf_file.name}")

        for excel_file in excel_files:
            dest = self.work_dir / excel_file.name
            shutil.copy2(excel_file, dest)
            self.uploaded_files.append(str(dest))
            print(f"📊 复制Excel: {excel_file.name}")

        self._categorize_files()
        return len(pdf_files) + len(excel_files)

    def _categorize_files(self):
        """分类文件"""
        self.pdf_files = []
        self.manual_files = []

        for file_path in self.uploaded_files:
            path = Path(file_path)
            if path.suffix.lower() == '.pdf':
                self.pdf_files.append(file_path)
            elif path.suffix.lower() in ['.xlsx', '.xls']:
                self.manual_files.append(file_path)

        print(f"\n📋 文件分类完成:")
        print(f"   PDF文件: {len(self.pdf_files)}个")
        print(f"   Manual文件: {len(self.manual_files)}个")

    def auto_match_files(self) -> List[Tuple[str, str, str]]:
        """自动匹配PDF和Manual文件"""
        matches = []

        for pdf_path in self.pdf_files:
            pdf_name = Path(pdf_path).stem

            # 尝试多种匹配模式
            potential_matches = []

            for manual_path in self.manual_files:
                manual_name = Path(manual_path).stem

                # 匹配模式1: 完全相同
                if pdf_name.lower() == manual_name.lower():
                    potential_matches.append((manual_path, 1.0, "完全匹配"))

                # 匹配模式2: PDF名称_manual
                elif manual_name.lower() == f"{pdf_name.lower()}_manual":
                    potential_matches.append((manual_path, 0.9, "后缀匹配"))

                # 匹配模式3: manual_PDF名称
                elif manual_name.lower() == f"manual_{pdf_name.lower()}":
                    potential_matches.append((manual_path, 0.9, "前缀匹配"))

                # 匹配模式4: 包含关系
                elif pdf_name.lower() in manual_name.lower() or manual_name.lower() in pdf_name.lower():
                    potential_matches.append((manual_path, 0.7, "部分匹配"))

            # 选择最佳匹配
            if potential_matches:
                best_match = max(potential_matches, key=lambda x: x[1])
                matches.append((pdf_path, best_match[0], pdf_name))
                print(f"✅ 匹配: {pdf_name} → {Path(best_match[0]).name} ({best_match[2]})")
            else:
                print(f"❌ 未找到匹配: {pdf_name}")

        self.file_pairs = matches
        return matches

    def manual_pair_files(self):
        """手动配对文件"""
        print("\n🔧 手动文件配对")
        print("=" * 40)

        self.file_pairs = []

        print("可用的PDF文件:")
        for i, pdf_path in enumerate(self.pdf_files, 1):
            print(f"  {i}. {Path(pdf_path).name}")

        print("\n可用的Manual文件:")
        for i, manual_path in enumerate(self.manual_files, 1):
            print(f"  {i}. {Path(manual_path).name}")

        for pdf_path in self.pdf_files:
            pdf_name = Path(pdf_path).name
            print(f"\n为PDF文件 '{pdf_name}' 选择Manual文件:")

            for i, manual_path in enumerate(self.manual_files, 1):
                print(f"  {i}. {Path(manual_path).name}")

            try:
                choice = int(input("请输入Manual文件编号 (0跳过): "))
                if choice > 0 and choice <= len(self.manual_files):
                    manual_path = self.manual_files[choice - 1]
                    doc_name = Path(pdf_path).stem
                    self.file_pairs.append((pdf_path, manual_path, doc_name))
                    print(f"✅ 配对成功: {pdf_name} → {Path(manual_path).name}")
                else:
                    print(f"⏭️ 跳过: {pdf_name}")
            except ValueError:
                print(f"⏭️ 输入无效，跳过: {pdf_name}")

    def validate_manual_files(self) -> Dict[str, bool]:
        """验证Manual文件格式"""
        validation_results = {}

        print("\n🔍 验证Manual文件格式...")

        for manual_path in self.manual_files:
            file_name = Path(manual_path).name
            try:
                df = pd.read_excel(manual_path)

                # 检查必需列
                required_columns = ['kpi_text']
                missing_columns = [col for col in required_columns if col not in df.columns]

                if missing_columns:
                    print(f"❌ {file_name}: 缺少列 {missing_columns}")
                    validation_results[manual_path] = False
                else:
                    # 检查数据
                    non_empty_rows = df['kpi_text'].notna().sum()
                    print(f"✅ {file_name}: {len(df)}行数据, {non_empty_rows}个有效KPI")
                    validation_results[manual_path] = True

            except Exception as e:
                print(f"❌ {file_name}: 读取失败 - {e}")
                validation_results[manual_path] = False

        return validation_results

    def show_file_summary(self):
        """显示文件汇总"""
        print(f"\n📊 文件上传汇总")
        print("=" * 40)
        print(f"总文件数: {len(self.uploaded_files)}")
        print(f"PDF文件: {len(self.pdf_files)}")
        print(f"Manual文件: {len(self.manual_files)}")
        print(f"配对文件: {len(self.file_pairs)}")

        if self.file_pairs:
            print(f"\n📋 配对结果:")
            for i, (pdf_path, manual_path, doc_name) in enumerate(self.file_pairs, 1):
                print(f"  {i}. {doc_name}")
                print(f"     PDF: {Path(pdf_path).name}")
                print(f"     Manual: {Path(manual_path).name}")

    def get_file_pairs_for_batch_processing(self) -> List[Tuple[str, str, str]]:
        """获取用于批量处理的文件对"""
        return self.file_pairs

    def create_batch_processor_from_uploads(self):
        """从上传的文件创建批量处理器"""
        if not self.file_pairs:
            print("❌ 没有可用的文件对")
            return None

        # 导入批量处理器
        from your_main_script import create_batch_processor  # 需要替换为实际的导入

        processor = create_batch_processor()

        for pdf_path, manual_path, doc_name in self.file_pairs:
            processor.add_file_pair(pdf_path, manual_path, doc_name)

        return processor

In [35]:
# ============ 便捷使用函数 ============

def create_batch_processor():
    """A convenience function to create a batch KPI processor"""
    return BatchKPIProcessor()

def quick_batch_from_directories(pdf_dir: str, manual_dir: str):
    """Quickly create a batch process from two directories"""
    processor = BatchKPIProcessor()

    # 添加文件对
    added_count = processor.add_multiple_pairs_from_directory(pdf_dir, manual_dir)

    if added_count == 0:
        print("❌ No matching PDF and Manual file pairs found.")
        return None

    # 显示文件列表
    processor.list_file_pairs()

    # 询问是否继续
    response = input(f"\nStart processing these {added_count} documents? (y/n): ")
    if response.lower() == 'y':
        processor.run_batch_processing()
        return processor
    else:
        print("Batch processing cancelled.")
        return processor

def manual_batch_setup():
    """Interactive function for manually configuring a batch process"""
    processor = BatchKPIProcessor()

    print("📋 Manual Batch Processing Setup")
    print("=" * 40)

    while True:
        print(f"\nCurrently added {len(processor.file_pairs)} file pair(s)")
        print("1. Add a single file pair")
        print("2. Add multiple file pairs from directories")
        print("3. View added file pairs")
        print("4. Start batch processing")
        print("5. Exit")

        choice = input("Select an option (1-5): ")

        if choice == '1':
            pdf_path = input("PDF file path: ")
            manual_path = input("Manual file path: ")
            doc_name = input("Document name (press Enter for default): ").strip()

            if not doc_name:
                doc_name = None

            processor.add_file_pair(pdf_path, manual_path, doc_name)

        elif choice == '2':
            pdf_dir = input("PDF directory: ")
            manual_dir = input("Manual directory: ")
            added = processor.add_multiple_pairs_from_directory(pdf_dir, manual_dir)
            print(f"{added} file pair(s) added.")

        elif choice == '3':
            processor.list_file_pairs()

        elif choice == '4':
            if processor.file_pairs:
                processor.run_batch_processing()
                break
            else:
                print("❌ No file pairs added.")

        elif choice == '5':
            print("Exiting batch setup.")
            break
        else:
            print("Invalid selection. Please try again.")

    return processor

In [36]:
# Enhanced comparison function for the main code
def enhanced_compare_with_manual_kpis(df_auto: pd.DataFrame, manual_xlsx_path: str,
                                     output_dir: str = "validation_results") -> Dict[str, any]:
    """
    增强版比较函数 - 显示详细的文本验证结果 + 元数据验证
    """
    if not Path(manual_xlsx_path).exists():
        logging.warning(f"Manual KPI file not found: {manual_xlsx_path}")
        return {}

    # Save auto KPIs to temporary file for validation pipeline
    temp_auto_path = Path(output_dir) / "temp_auto_kpis.xlsx"
    temp_auto_path.parent.mkdir(exist_ok=True)
    df_auto.to_excel(temp_auto_path, index=False)

    try:
        # Step 1: 运行详细的文本验证
        print(f"\n🔍 Running detailed text validation against {Path(manual_xlsx_path).name}...")

        validator = KPIValidationPipeline(
            manual_excel_path=manual_xlsx_path,
            auto_excel_path=str(temp_auto_path),
            output_dir=output_dir
        )

        # Run full validation
        text_validation_results = validator.run_full_validation()

        # 显示详细的文本验证摘要
        if text_validation_results and 'best_metrics' in text_validation_results:
            best_metrics = text_validation_results['best_metrics']

            print("\n" + "="*60)
            print("KPI EXTRACTION VALIDATION SUMMARY")
            print("="*60)
            print(f"📊 Dataset: {len(validator.manual_df)} manual vs {len(validator.auto_df)} auto KPIs")
            print(f"🎯 Best F1 Score: {best_metrics['f1_score']:.3f}")
            print(f"📈 Precision: {best_metrics['precision']:.3f}")
            print(f"📉 Recall: {best_metrics['recall']:.3f}")
            print(f"✅ True Positives: {best_metrics['true_positives']}")
            print(f"❌ False Positives: {best_metrics['false_positives']}")
            print(f"⚠️  False Negatives: {best_metrics['false_negatives']}")
            print("="*60)
            print(f"📁 Results saved to: {output_dir}")
            print("="*60)

        # Step 2: 运行元数据验证
        print(f"\n📊 Running metadata validation...")

        metadata_validator = MetadataValidationExtension(validator)
        metadata_results = metadata_validator.validate_metadata_for_matched_pairs()

        # 保存元数据验证结果
        metadata_saved_files = metadata_validator.save_metadata_validation_results(f"{output_dir}/metadata_validation")

        # 显示元数据验证摘要
        if metadata_results:
            overall_scores = metadata_results.get('overall_scores', {})

            print(f"\n📋 METADATA VALIDATION SUMMARY")
            print("="*60)
            print(f"📊 Metadata Overall Score: {overall_scores.get('overall_metadata_score', 0):.3f}")
            print(f"🎯 Validated KPI Pairs: {metadata_results.get('total_pairs', 0)}")

            print(f"\n📂 Metadata Category Scores:")
            for category, score in overall_scores.items():
                if category.endswith('_fields'):
                    category_name = category.replace('_', ' ').title()
                    print(f"   {category_name}: {score:.3f}")

            # 显示问题字段
            if metadata_results.get('problematic_field_analysis'):
                print(f"\n⚠️  Fields Needing Attention:")
                for field_name, analysis in metadata_results['problematic_field_analysis'].items():
                    priority_emoji = "🔴" if analysis['improvement_priority'] == 'high' else "🟡" if analysis['improvement_priority'] == 'medium' else "🟢"
                    print(f"   {priority_emoji} {field_name}: {analysis['accuracy']:.3f} ({analysis['improvement_priority']} priority)")

            print("="*60)
            print(f"📁 Metadata results saved to: {output_dir}/metadata_validation")
            print("="*60)

        # 组合结果
        complete_results = {
            'text_validation': text_validation_results,
            'metadata_validation': metadata_results,
            'saved_files': {
                'text_validation_files': text_validation_results.get('saved_files', {}),
                'metadata_validation_files': metadata_saved_files
            }
        }

        # Clean up temporary file
        if temp_auto_path.exists():
            temp_auto_path.unlink()

        return complete_results

    except Exception as e:
        logging.error(f"Enhanced validation failed: {e}")
        if temp_auto_path.exists():
            temp_auto_path.unlink()
        return {}


# Integration function for the main pipeline
def run_kpi_extraction_with_validation():
    """Run KPI extraction with comprehensive validation"""
    print("🚀 Starting KPI extraction with automated validation...")

    # Validate environment
    if not validate_environment():
        print("Please fix the environment issues before running.")
        return None

    try:
        # Step 1: Run KPI extraction
        print("\n📊 Step 1: Extracting KPIs...")
        df_auto = process_sustainability_report_with_enhanced_images(PDF_PATH)

        # Save auto results
        save_results(df_auto, EXPORT_AUTO_XLSX, PDF_PATH)
        print(f"✅ Extracted {len(df_auto)} KPIs and saved to {EXPORT_AUTO_XLSX}")

        # Step 2: Run validation if manual file exists
        if MANUAL_XLSX and Path(MANUAL_XLSX).exists():
            print(f"\n🔍 Step 2: Running validation against {MANUAL_XLSX}...")
            validation_results = enhanced_compare_with_manual_kpis(
                df_auto, MANUAL_XLSX, "validation_results"
            )

            if validation_results:
                best_metrics = validation_results['best_metrics']
                print(f"\n🎯 Validation completed!")
                print(f"   F1 Score: {best_metrics['f1_score']:.3f}")
                print(f"   Precision: {best_metrics['precision']:.3f}")
                print(f"   Recall: {best_metrics['recall']:.3f}")

                return {
                    'extracted_kpis': df_auto,
                    'validation_results': validation_results
                }
            else:
                print("⚠️ Validation failed, but extraction completed successfully")
                return {'extracted_kpis': df_auto}
        else:
            print(f"\n⚠️ Manual KPI file not found ({MANUAL_XLSX}), skipping validation")
            return {'extracted_kpis': df_auto}

    except Exception as e:
        print(f"❌ Pipeline failed: {e}")
        import traceback
        traceback.print_exc()
        return None


# Batch validation function for multiple documents
def run_batch_validation(pdf_list: List[str], manual_list: List[str],
                        output_base_dir: str = "batch_validation"):
    """
    Run validation across multiple PDF documents

    Args:
        pdf_list: List of PDF file paths
        manual_list: List of corresponding manual annotation files
        output_base_dir: Base directory for validation results
    """
    batch_results = []

    for i, (pdf_path, manual_path) in enumerate(zip(pdf_list, manual_list)):
        print(f"\n{'='*60}")
        print(f"Processing document {i+1}/{len(pdf_list)}: {Path(pdf_path).name}")
        print(f"{'='*60}")

        try:
            # Set up paths for this document
            doc_name = Path(pdf_path).stem
            doc_output_dir = Path(output_base_dir) / doc_name
            doc_output_dir.mkdir(parents=True, exist_ok=True)

            # Extract KPIs
            global PDF_PATH
            original_pdf_path = PDF_PATH
            PDF_PATH = pdf_path

            df_auto = process_sustainability_report_with_enhanced_images(PDF_PATH)
            auto_excel_path = doc_output_dir / f"{doc_name}_auto_kpis.xlsx"
            save_results(df_auto, str(auto_excel_path), PDF_PATH)

            # Run validation
            validation_results = enhanced_compare_with_manual_kpis(
                df_auto, manual_path, str(doc_output_dir / "validation")
            )

            # Store results
            doc_result = {
                'document': doc_name,
                'pdf_path': pdf_path,
                'manual_path': manual_path,
                'extracted_kpis': len(df_auto),
                'validation_results': validation_results.get('best_metrics', {}),
                'output_dir': str(doc_output_dir)
            }
            batch_results.append(doc_result)

            # Restore original PDF path
            PDF_PATH = original_pdf_path

        except Exception as e:
            print(f"❌ Failed to process {doc_name}: {e}")
            batch_results.append({
                'document': doc_name,
                'pdf_path': pdf_path,
                'manual_path': manual_path,
                'error': str(e)
            })

    # Generate batch summary
    batch_summary_path = Path(output_base_dir) / "batch_summary.xlsx"
    batch_df = pd.DataFrame(batch_results)
    batch_df.to_excel(batch_summary_path, index=False)

    print(f"\n🎉 Batch validation completed!")
    print(f"📊 Processed {len(pdf_list)} documents")
    print(f"📁 Results saved to {output_base_dir}")
    print(f"📋 Summary available at {batch_summary_path}")

    return batch_results


# Quick validation function for testing
def quick_validation_test(manual_xlsx: str = None, auto_xlsx: str = None):
    """Quick validation test with existing files"""
    manual_file = manual_xlsx or MANUAL_XLSX
    auto_file = auto_xlsx or EXPORT_AUTO_XLSX

    if not Path(manual_file).exists():
        print(f"❌ Manual file not found: {manual_file}")
        return None

    if not Path(auto_file).exists():
        print(f"❌ Auto file not found: {auto_file}")
        return None

    print(f"🔍 Quick validation test:")
    print(f"  Manual: {manual_file}")
    print(f"  Auto: {auto_file}")

    try:
        validator = KPIValidationPipeline(
            manual_excel_path=manual_file,
            auto_excel_path=auto_file,
            output_dir="quick_validation"
        )

        results = validator.run_full_validation()
        return results

    except Exception as e:
        print(f"❌ Quick validation failed: {e}")
        return None


# Performance benchmarking function
def benchmark_extraction_methods():
    """Benchmark different extraction methods with validation"""
    methods = {
        'text_only': process_text_only,
        'with_images': process_sustainability_report_with_enhanced_images,
        'optimized': process_sustainability_report_OPTIMIZED
    }

    benchmark_results = {}

    for method_name, method_func in methods.items():
        print(f"\n🧪 Benchmarking {method_name}...")

        try:
            import time
            start_time = time.time()

            # Run extraction
            df_result = method_func(PDF_PATH)
            extraction_time = time.time() - start_time

            # Save results
            method_output = f"{method_name}_{EXPORT_AUTO_XLSX}"
            save_results(df_result, method_output, PDF_PATH)

            # Run validation if manual file exists
            validation_metrics = {}
            if MANUAL_XLSX and Path(MANUAL_XLSX).exists():
                validation_results = enhanced_compare_with_manual_kpis(
                    df_result, MANUAL_XLSX, f"benchmark_{method_name}"
                )
                if validation_results:
                    validation_metrics = validation_results['best_metrics']

            benchmark_results[method_name] = {
                'extraction_time': extraction_time,
                'kpi_count': len(df_result),
                'kpis_per_second': len(df_result) / extraction_time,
                'validation_metrics': validation_metrics
            }

            print(f"✅ {method_name}: {len(df_result)} KPIs in {extraction_time:.1f}s")
            if validation_metrics:
                print(f"   F1: {validation_metrics.get('f1_score', 0):.3f}")

        except Exception as e:
            print(f"❌ {method_name} failed: {e}")
            benchmark_results[method_name] = {'error': str(e)}

    # Save benchmark results
    benchmark_df = pd.DataFrame(benchmark_results).T
    benchmark_df.to_excel("extraction_benchmark.xlsx")

    print(f"\n🏆 Benchmark completed!")
    print(f"📊 Results saved to extraction_benchmark.xlsx")

    return benchmark_results


# Usage examples and documentation
def validation_usage_examples():
    """Show usage examples for the validation pipeline"""
    print("""
# KPI Validation Pipeline Usage Examples

## 1. Basic validation with existing files
```python
validator = KPIValidationPipeline(
    manual_excel_path="manual_kpis.xlsx",
    auto_excel_path="auto_kpis.xlsx"
)
results = validator.run_full_validation()
```

## 2. Integrated extraction + validation
```python
results = run_kpi_extraction_with_validation()
```

## 3. Quick validation test
```python
results = quick_validation_test("manual.xlsx", "auto.xlsx")
```

## 4. Batch validation for multiple documents
```python
pdf_files = ["doc1.pdf", "doc2.pdf", "doc3.pdf"]
manual_files = ["manual1.xlsx", "manual2.xlsx", "manual3.xlsx"]
batch_results = run_batch_validation(pdf_files, manual_files)
```

## 5. Benchmark different extraction methods
```python
benchmark_results = benchmark_extraction_methods()
```

## 6. Custom threshold analysis
```python
validator = KPIValidationPipeline("manual.xlsx", "auto.xlsx")
validator.run_comprehensive_evaluation()

# Check performance at different thresholds
for threshold in [0.5, 0.7, 0.9]:
    metrics = validator.calculate_metrics_at_threshold(threshold)
    print(f"Threshold {threshold}: F1={metrics['f1_score']:.3f}")
```

## Output Files Generated:
- validation_results.json - Complete results in JSON format
- detailed_matches.xlsx - All matched KPIs with similarity scores
- error_analysis.xlsx - False positives and false negatives
- validation_report.md - Human-readable report
- threshold_analysis.xlsx - Performance across different thresholds
- validation_visualizations.png - Comprehensive charts and graphs

## Key Metrics Explained:
- **Precision**: % of auto KPIs that match manual annotations
- **Recall**: % of manual KPIs found by automatic extraction
- **F1 Score**: Harmonic mean of precision and recall
- **True Positives**: Correctly identified KPIs
- **False Positives**: Auto KPIs not in manual annotations
- **False Negatives**: Manual KPIs missed by extraction
""")

In [37]:
# 修改主函数，显示完整的详细信息
def enhanced_main_with_detailed_output():
    """增强版main函数，显示详细的验证输出"""
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s: %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S"
    )

    try:
        if not os.path.exists(PDF_PATH):
            logging.error(f"PDF file not found: {PDF_PATH}")
            return

        # Step 1: KPI提取
        print(f"\n🚀 Starting KPI extraction from {Path(PDF_PATH).name}...")
        df_auto = process_sustainability_report_with_enhanced_images(PDF_PATH)
        save_results(df_auto, EXPORT_AUTO_XLSX, PDF_PATH)
        print(f"✅ KPI extraction completed: {len(df_auto)} KPIs extracted")

        # Step 2: 详细验证（如果有manual文件）
        if MANUAL_XLSX and Path(MANUAL_XLSX).exists():
            validation_start_time = time.time()

            print(f"\n🔍 Starting comprehensive validation against {Path(MANUAL_XLSX).name}...")
            complete_validation_results = enhanced_compare_with_manual_kpis(
                df_auto, MANUAL_XLSX, "comprehensive_validation"
            )

            validation_time = time.time() - validation_start_time

            if complete_validation_results:
                # 显示综合摘要
                text_metrics = complete_validation_results['text_validation'].get('best_metrics', {})
                metadata_scores = complete_validation_results['metadata_validation'].get('overall_scores', {})

                print(f"\n🎯 Verification completed:")
                print(f"   Text F1 Score: {text_metrics.get('f1_score', 0):.3f}")
                print(f"   Text Precision: {text_metrics.get('precision', 0):.3f}")
                print(f"   Text Recall: {text_metrics.get('recall', 0):.3f}")
                print(f"   Metadata Overall Score: {metadata_scores.get('overall_metadata_score', 0):.3f}")
                print(f"⏱️  Validation time: {validation_time:.1f}秒")
                print(f"📁 Complete results saved to: comprehensive_validation/")

            else:
                print("⚠️ Validation encountered issues, but extraction completed successfully")
        else:
            logging.info("Manual KPI file not found, skipping validation.")

        # Step 3: 显示提取摘要
        if not df_auto.empty:
            print(f"\n=== EXTRACTION SUMMARY ===")
            print(f"Total KPIs extracted: {len(df_auto)}")

            if 'source_type' in df_auto.columns:
                source_counts = df_auto['source_type'].value_counts()
                print(f"From text/tables: {source_counts.get('text', 0)}")
                print(f"From images/charts: {source_counts.get('image', 0)}")

            if 'kpi_theme' in df_auto.columns:
                theme_counts = df_auto['kpi_theme'].value_counts()
                print(f"\nKPI Distribution by Theme:")
                for theme, count in theme_counts.items():
                    print(f"  {theme}: {count}")
        else:
            print("\nNo KPIs were extracted from the document.")

    except Exception as e:
        logging.error(f"Error in enhanced main execution: {e}")
        import traceback
        traceback.print_exc()

In [39]:
# 创建详细输出的批量处理器
def create_batch_processor_with_detailed_output():
    """创建支持详细输出的批量处理器"""
    return BatchKPIProcessorWithDetailedOutput()

# 修改集成执行函数
def integrated_main_execution_detailed():
    """集成的主执行函数 - 详细输出版本"""
    print("🚀 Enhanced KPI Extraction System - Detailed Output")
    print("=" * 60)
    print("1. Single PDF processing (detailed text + metadata validation)")
    print("2. Batch PDF processing (detailed text + metadata validation)")
    print("3. Quick batch from directories (detailed output)")
    print("4. Quick metadata validation test")
    print("5. View usage examples")

    try:
        choice = input("Please select processing mode (1-5): ")

        if choice == '1':
            # 单个PDF处理，详细输出
            enhanced_main_with_detailed_output()

        elif choice == '2':
            # 批量处理，详细输出
            processor = create_batch_processor_with_detailed_output()

            # 让用户手动添加文件对
            while True:
                pdf_path = input("Enter PDF file path (press Enter to finish): ").strip()
                if not pdf_path:
                    break
                manual_path = input("Enter corresponding Manual file path: ").strip()
                doc_name = input("Document name (press Enter for default): ").strip() or None

                processor.add_file_pair(pdf_path, manual_path, doc_name)

            if processor.file_pairs:
                processor.list_file_pairs()
                confirm = input(f"\nStart detailed processing of these {len(processor.file_pairs)} documents? (y/n): ")
                if confirm.lower() == 'y':
                    processor.run_batch_processing()
                else:
                    print("Batch processing cancelled.")
            else:
                print("❌ No file pairs added.")

        elif choice == '3':
            # 快速目录批量处理，详细输出
            pdf_dir = input("PDF files directory path: ").strip()
            manual_dir = input("Manual files directory path: ").strip()

            processor = create_batch_processor_with_detailed_output()
            added_count = processor.add_multiple_pairs_from_directory(pdf_dir, manual_dir)

            if added_count == 0:
                print("❌ No matching PDF and Manual file pairs found.")
                return None

            processor.list_file_pairs()
            response = input(f"\nStart detailed processing of these {added_count} documents? (y/n): ")
            if response.lower() == 'y':
                processor.run_batch_processing()
            else:
                print("Batch processing cancelled.")

        elif choice == '4':
            # 快速元数据验证测试
            quick_test_metadata_validation()

        elif choice == '5':
            # 显示使用示例
            metadata_validation_usage_examples()

        else:
            print("Invalid selection, running default single PDF processing")
            enhanced_main_with_detailed_output()

    except KeyboardInterrupt:
        print("\nUser cancelled operation")
    except Exception as e:
        print(f"Execution error: {e}")
        print("Running default single PDF processing")
        enhanced_main_with_detailed_output()

# 更新主执行部分
if __name__ == "__main__":
    # 使用详细输出的集成执行函数
    integrated_main_execution_detailed()

🚀 Enhanced KPI Extraction System - Detailed Output
1. Single PDF processing (detailed text + metadata validation)
2. Batch PDF processing (detailed text + metadata validation)
3. Quick batch from directories (detailed output)
4. Quick metadata validation test
5. View usage examples
Please select processing mode (1-5): 2
Enter PDF file path (press Enter to finish): /content/NewNew.pdf
Enter corresponding Manual file path: /content/NewNew.xlsx
Document name (press Enter for default): NewNew
Enter PDF file path (press Enter to finish): /content/fujit.pdf
Enter corresponding Manual file path: /content/fujit.xlsx
Document name (press Enter for default): fujit
Enter PDF file path (press Enter to finish): /content/Cineplex.pdf
Enter corresponding Manual file path: /content/Cineplex.xlsx
Document name (press Enter for default): Cineplex
Enter PDF file path (press Enter to finish): 

📋 已添加的文件对 (共 3 对):
--------------------------------------------------------------------------------
 1. 文档: NewN

✅ Extraction completed: 55 KPIs

🔍 Step 2: Running comprehensive validation against NewNew.xlsx...

🔍 Running detailed text validation against NewNew.xlsx...

KPI EXTRACTION VALIDATION SUMMARY
📊 Dataset: 46 manual vs 55 auto KPIs
🎯 Best F1 Score: 0.911
📈 Precision: 0.836
📉 Recall: 1.000
✅ True Positives: 46
❌ False Positives: 9
⚠️  False Negatives: 0
📁 Results saved to: batch_kpi_results/batch_20250731_110839/doc_1_NewNew/validation

KPI EXTRACTION VALIDATION SUMMARY
📊 Dataset: 46 manual vs 55 auto KPIs
🎯 Best F1 Score: 0.911
📈 Precision: 0.836
📉 Recall: 1.000
✅ True Positives: 46
❌ False Positives: 9
⚠️  False Negatives: 0
📁 Results saved to: batch_kpi_results/batch_20250731_110839/doc_1_NewNew/validation

📊 Running metadata validation...

📋 METADATA VALIDATION SUMMARY
📊 Metadata Overall Score: 0.925
🎯 Validated KPI Pairs: 46

📂 Metadata Category Scores:
   Document Fields: 1.000
   Classification Fields: 1.000
   Quantitative Fields: 0.748
   Analysis Fields: 0.986

⚠️  Fields Needin


```json
[
    {
        "kpi_text": "In FY...

```json
[]
```

The text provided does not...


✅ Extraction completed: 31 KPIs

🔍 Step 2: Running comprehensive validation against fujit.xlsx...

🔍 Running detailed text validation against fujit.xlsx...

KPI EXTRACTION VALIDATION SUMMARY
📊 Dataset: 12 manual vs 31 auto KPIs
🎯 Best F1 Score: 0.326
📈 Precision: 0.226
📉 Recall: 0.583
✅ True Positives: 7
❌ False Positives: 24
⚠️  False Negatives: 5
📁 Results saved to: batch_kpi_results/batch_20250731_110839/doc_2_fujit/validation

KPI EXTRACTION VALIDATION SUMMARY
📊 Dataset: 12 manual vs 31 auto KPIs
🎯 Best F1 Score: 0.326
📈 Precision: 0.226
📉 Recall: 0.583
✅ True Positives: 7
❌ False Positives: 24
⚠️  False Negatives: 5
📁 Results saved to: batch_kpi_results/batch_20250731_110839/doc_2_fujit/validation

📊 Running metadata validation...

📋 METADATA VALIDATION SUMMARY
📊 Metadata Overall Score: 0.945
🎯 Validated KPI Pairs: 7

📂 Metadata Category Scores:
   Document Fields: 1.000
   Classification Fields: 1.000
   Quantitative Fields: 0.807
   Analysis Fields: 1.000

⚠️  Fields Needing Att

✅ Extraction completed: 4 KPIs

🔍 Step 2: Running comprehensive validation against Cineplex.xlsx...

🔍 Running detailed text validation against Cineplex.xlsx...


ERROR:root:PDF file not found: /content/Test_Unknown_northwest-sustainability-report-2022_fbqow68f-60-74.pdf



KPI EXTRACTION VALIDATION SUMMARY
📊 Dataset: 6 manual vs 4 auto KPIs
🎯 Best F1 Score: 0.800
📈 Precision: 1.000
📉 Recall: 0.667
✅ True Positives: 4
❌ False Positives: 0
⚠️  False Negatives: 2
📁 Results saved to: batch_kpi_results/batch_20250731_110839/doc_3_Cineplex/validation

KPI EXTRACTION VALIDATION SUMMARY
📊 Dataset: 6 manual vs 4 auto KPIs
🎯 Best F1 Score: 0.800
📈 Precision: 1.000
📉 Recall: 0.667
✅ True Positives: 4
❌ False Positives: 0
⚠️  False Negatives: 2
📁 Results saved to: batch_kpi_results/batch_20250731_110839/doc_3_Cineplex/validation

📊 Running metadata validation...

📋 METADATA VALIDATION SUMMARY
📊 Metadata Overall Score: 0.835
🎯 Validated KPI Pairs: 4

📂 Metadata Category Scores:
   Document Fields: 1.000
   Classification Fields: 0.430
   Quantitative Fields: 0.914
   Analysis Fields: 1.000

⚠️  Fields Needing Attention:
   🟢 Absolute Page Number: 1.000 (low priority)
   🟡 kpi_theme: 0.500 (medium priority)
   🟢 quantitative_value: 1.000 (low priority)
📁 Metadata res